In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

service = Service(ChromeDriverManager().install())  # 自动安装对应版本的chromedriver
driver = webdriver.Chrome(service=service)
service.start()

# 设置 ChromeOptions
options = Options()

# 初始化 WebDriver
driver = WebDriver(service=service, options=options)

# 打开 ArchDaily 网站
driver.get('https://www.archdaily.com/search/projects/categories/library')

# 在搜索框中输入关键词
search_box = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'searchbar-input')))
search_box.send_keys('university')

# 在搜索框中按回车键
search_box.send_keys(Keys.RETURN)

# 找到按钮并点击
button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//div[@data-insights-category="search-layout-toggler"]')))
button.click()

import time

# 循环滚动到页面的底部，直到页面没有新的内容被加载
while True:
    # 记录滚动前的页面高度
    old_scroll_height = driver.execute_script("return document.body.scrollHeight")

    # 滚动到页面的底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 等待一段时间让页面加载
    time.sleep(5)

    # 记录滚动后的页面高度
    new_scroll_height = driver.execute_script("return document.body.scrollHeight")

    # 如果滚动前后页面的高度相同，说明没有新的内容被加载，退出循环
    if new_scroll_height == old_scroll_height:
        break

print("page loaded successfully")

# # 等待搜索结果加载完成
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'afd-search-result')))

# print("search result loaded successfully")

# 获取搜索结果中所有项目的链接
project_links_elements = driver.find_elements(By.CLASS_NAME, 'afd-title--black-link')

# 获取每个链接元素的 href 属性
project_links = [link.get_attribute('href') for link in project_links_elements]

# 打印所有的链接
for link in project_links:
    print(link)

# 关闭浏览器
driver.quit()



page loaded successfully
https://www.archdaily.com/1024858/sona-university-centre-and-library-block-imk-architects?ad_source=search&ad_medium=projects_tab&ad_source=search&ad_medium=search_result_projects
https://www.archdaily.com/1020833/the-atrium-university-library-scau?ad_source=search&ad_medium=projects_tab&ad_source=search&ad_medium=search_result_projects
https://www.archdaily.com/42319/university-library-of-the-university-of-amsterdam-roelof-mulder-ira-koers?ad_source=search&ad_medium=projects_tab&ad_source=search&ad_medium=search_result_projects
https://www.archdaily.com/557018/university-library-a02-atelier?ad_source=search&ad_medium=projects_tab&ad_source=search&ad_medium=search_result_projects
https://www.archdaily.com/805685/the-library-of-shandong-normal-university-the-architectural-design-and-research-institute-of-zhejiang-university-uad-the-7th-architectural-design-and-research-institute?ad_source=search&ad_medium=projects_tab&ad_source=search&ad_medium=search_result_pro

In [1]:
keywords = [
    "Science", "Technology", "Innovation", 
    "Interactive", "Learning", "Planetarium", "Lab", "Discovery", 
    "Nature", "Earth", "Biology", "Geology", 
    "Paleontology", "Anthropology", "Botanical", "Zoology", "Astronomy", 
    "Marine", "Ecology", "Physics", "Chemistry", "Meteorology", "Robot"
]

base_url = "https://www.archdaily.com/search/projects/categories/museum"
urls= []
for keyword in keywords:
    urls.append(f"{base_url}?q={keyword}")

urls.append("https://www.archdaily.com/search/projects/categories/science-center")
urls.append("https://www.archdaily.com/search/projects/categories/planetarium")
print(urls)

    

['https://www.archdaily.com/search/projects/categories/museum?q=Science', 'https://www.archdaily.com/search/projects/categories/museum?q=Technology', 'https://www.archdaily.com/search/projects/categories/museum?q=Innovation', 'https://www.archdaily.com/search/projects/categories/museum?q=Interactive', 'https://www.archdaily.com/search/projects/categories/museum?q=Learning', 'https://www.archdaily.com/search/projects/categories/museum?q=Planetarium', 'https://www.archdaily.com/search/projects/categories/museum?q=Lab', 'https://www.archdaily.com/search/projects/categories/museum?q=Discovery', 'https://www.archdaily.com/search/projects/categories/museum?q=Nature', 'https://www.archdaily.com/search/projects/categories/museum?q=Earth', 'https://www.archdaily.com/search/projects/categories/museum?q=Biology', 'https://www.archdaily.com/search/projects/categories/museum?q=Geology', 'https://www.archdaily.com/search/projects/categories/museum?q=Paleontology', 'https://www.archdaily.com/search/p

## new project list generater

In [ ]:
from selenium import webdriver 
from selenium.webdriver.common.by  import By 
from selenium.webdriver.support.ui  import WebDriverWait 
from selenium.webdriver.support  import expected_conditions as EC 
from selenium.webdriver.chrome.service  import Service 
from selenium.webdriver.chrome.options  import Options 
from selenium.common.exceptions  import TimeoutException 
from webdriver_manager.chrome  import ChromeDriverManager 
import pandas as pd 
import concurrent.futures  
import threading 
import time 
 
# 线程安全的DataFrame存储 
class ThreadSafeDataFrame:
    def __init__(self):
        self._lock = threading.Lock()
        self._df = pd.DataFrame()
        
    def append(self, new_df):
        with self._lock:
            self._df = pd.concat([self._df,  new_df])
            
    def get(self):
        with self._lock:
            return self._df.copy() 
 
# 全局线程安全对象 
shared_projects = ThreadSafeDataFrame()
 
# 每个线程的爬取任务 
def crawl_task(url):
    # 每个线程创建独立的driver实例 
    service = Service(ChromeDriverManager().install())
    service.start() 
    
    options = Options()
    options.add_argument("--headless")   # 建议添加无头模式 
    options.add_argument("--disable-gpu") 
    options.add_argument("--no-sandbox") 
    
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        driver.get(url) 
        
        # 点击布局切换按钮 
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH,  '//div[@data-insights-category="search-layout-toggler"]'))
        ).click()
        
        # 动态滚动加载 
        last_height = driver.execute_script("return  document.body.scrollHeight") 
        while True:
            driver.execute_script("window.scrollTo(0,  document.body.scrollHeight);") 
            time.sleep(2) 
            new_height = driver.execute_script("return  document.body.scrollHeight") 
            if new_height == last_height:
                break 
            last_height = new_height 
        
        # 解析数据 
        project_links = [link.get_attribute('href') 
                       for link in driver.find_elements(By.CLASS_NAME,  'afd-title--black-link')]
        
        if not project_links:
            print(f"No results found for {url}")
            return None 
            
        new_projects = pd.DataFrame({
            'project_id': [x.split('/')[3] for x in project_links],
            'link': project_links,
            'keyword': url.split('/')[-1], 
            'status': ''
        })
        
        # 线程安全的数据追加 
        shared_projects.append(new_projects) 
        
    except TimeoutException:
        print(f"Timeout occurred for {url}")
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
    finally:
        driver.quit() 
        service.stop() 
        
    return url 
 
# 主逻辑 
def main(urls):
    # 使用线程池（建议根据硬件调整线程数）
    with concurrent.futures.ThreadPoolExecutor(max_workers=10)  as executor:
        futures = {executor.submit(crawl_task,  url): url for url in urls}
        
        for future in concurrent.futures.as_completed(futures): 
            url = futures[future]
            try:
                result = future.result() 
                if result:
                    print(f"Completed: {result}")
            except Exception as e:
                print(f"Exception in {url}: {str(e)}")
    
    # 数据处理 
    projects = shared_projects.get() 
    
    if projects.empty: 
        print("No results found for any URL")
        return 
    
    # 去重处理 
    projects = projects.drop_duplicates('project_id',  keep='first')
    
    # 标记逻辑 
    projects.loc[projects['link'].str.split('-').apply(lambda  x: 'art' in x), 'status'] = 'delete'
    
    print(projects)
    return projects 
 
if __name__ == "__main__":
    
    result_df = main(urls)

Completed: https://www.archdaily.com/search/projects/categories/museum?q=Planetarium
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Lab
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Discovery
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Earth
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Learning
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Innovation
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Technology
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Science
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Biology
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Interactive
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Geology
Completed: https://www.archdaily.com/search/projects/categories/museum?q=Paleontology
Completed: h

In [3]:
# write to csv
result_df.to_csv('archdaily_projects.csv', index=False)

In [3]:
import csv
import os
import requests
import time
import re
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains

service = Service(ChromeDriverManager().install())  # 自动安装对应版本的chromedriver
service.start()

# 设置 ChromeOptions
options = Options()
options.add_argument('--lang=en')  # 设置语言为英语

# 初始化 WebDriver
driver = WebDriver(service=service, options=options)
actions = ActionChains(driver)

def project_scraper(driver, project_link):
    try:
        response = requests.get(project_link)
        response.raise_for_status()  # 如果请求失败抛出异常
    except requests.exceptions.RequestException as e:
        print(f"Network error while fetching {project_link}: {e}")
        return "error"
    
    try:
        soup = BeautifulSoup(response.content, 'html.parser')
        print('Project details are being scraped.')
        paragraph = soup.find_all('p')
        list_of_paragraph = [each_paragpraph.text.strip() for each_paragpraph in paragraph]
        driver.get(project_link)

        timeout = 20
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "js-char-list", " " ))]')))
        
        try:    
            area = driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "afd-specs__item", " " )) and (((count(preceding-sibling::*) + 1) = 2) and parent::*)]')[0].text.split(':')[1].strip()
            year = driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "afd-specs__item", " " )) and (((count(preceding-sibling::*) + 1) = 3) and parent::*)]')[0].text.split(':')[1].strip()
            photographs = driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "afd-specs__item", " " )) and (((count(preceding-sibling::*) + 1) = 4) and parent::*)]')[0].text.split(':')[1].strip()
        except IndexError:
            print(f"Missing required elements on {project_link}")
            return "incomplete"

        project_dict = {
            'Project ID': project_link[26:32],
            'Project Title': driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "afd-title-big--bmargin-big", " " ))]')[0].text.split('/')[0].strip(),
            'Project Type': driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "afd-specs__header-category", " " ))]//a')[0].text.strip(),
            'City': driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "afd-specs__header-location", " " ))]')[0].text.split(',')[0].strip(),
            'Country': driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "afd-specs__header-location", " " ))]//a')[0].text.strip(),
            'Architects': driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "afd-specs__item", " " )) and (((count(preceding-sibling::*) + 1) = 1) and parent::*)]')[0].text.split(':')[1].strip(),
            'Area': area,
            'Year': year,
            'Photographs': photographs,
            'Project URL': project_link.strip(),
            'Text Content': list_of_paragraph[7:-2]
        }

        project_js = json.dumps(project_dict, indent=4, ensure_ascii=False).encode('utf8')
        project_code = project_link.split('/')[3]
        
        # 写入 JSON 文件
        with open(f'{project_code}.json', 'w', encoding='utf-8') as file:
            file.write(project_js.decode())
        
        print('Project details are ready.')
        return "downloaded"
    except Exception as e:
        print(f"Error while processing {project_link}: {e}")
        return "error"

def project_image_downloader(driver, project_link):
    try:
        response = requests.get(project_link)
        soup = BeautifulSoup(response.content, 'html.parser')
        a_tag = soup.find('a', attrs={'class': 'gallery-link afd-desktop-e'})
        link = base_url + a_tag.get('href')
        driver.get(link)

        timeout = 20
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "js-gal-mob-img-onview", " " ))]')))

        this_table = driver.find_elements(By.ID, 'gallery-items')
        for this in this_table:
            temp = this.get_attribute('data-images')

        large_urls = []
        urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', temp)
        for url in urls:
            if 'large_jpg' in str(url):
                large_urls.append(url)
        
        i = 1
        project_code = project_link.split('/')[3]

        if not os.path.exists(project_code):
            os.makedirs(project_code)
        
        for each in large_urls:
            with open(os.path.join(project_code, f'{i}.jpg'), 'wb') as handle:
                response = requests.get(each, stream=True)
                print(f'Picture number {i} downloaded')
                time.sleep(2)

                if not response.ok:
                    print(response)

                for block in response.iter_content(1024):
                    if not block:
                        break

                    handle.write(block)
            i += 1
    except Exception as e:
        print(f"Error downloading images for {project_link}: {e}")

def update_csv_status(csv_file, project_code, status):
    # Read CSV and update status column
    rows = []
    with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        header = next(reader)
        status_index = header.index('status')  # 获取status列的索引
        rows.append(header)
        for row in reader:
            if row[0] == project_code:
                row[status_index] = status  # Update the status
            rows.append(row)

    # Write the updated content back to CSV
    with open(csv_file, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(rows)

if __name__ == "__main__":
    csv_file = './archdaily_projects.csv'
    base_url = 'https://www.archdaily.com/'
    
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        header = next(reader)
        status_index = header.index('status')
        
        for row in reader:
            project_code = row[0]
            status = row[status_index]
            
            # Skip projects with status "downloaded", "error", or "incomplete"
            if status in ['downloaded', 'error', 'incomplete', 'duplicate', 'delete']:
                print(f"Skipping project {project_code} as it is marked as {status}.")
                continue
            
            project_link = base_url + project_code
            print(f"Scraping project: {project_link}")
            status = project_scraper(driver, project_link)
            update_csv_status(csv_file, project_code, status)
            
            # Only download images if status is not "incomplete"
            if status != "incomplete" and status != "error":
                project_image_downloader(driver, project_link)


Skipping project 904658 as it is marked as downloaded.
Skipping project 965203 as it is marked as downloaded.
Scraping project: https://www.archdaily.com/343719
Project details are being scraped.
Error while processing https://www.archdaily.com/343719: Message: 
Stacktrace:
	GetHandleVerifier [0x00BF74A3+25091]
	(No symbol) [0x00B7DC04]
	(No symbol) [0x00A5B373]
	(No symbol) [0x00A9F4DC]
	(No symbol) [0x00A9F65B]
	(No symbol) [0x00ADD8E2]
	(No symbol) [0x00AC1F54]
	(No symbol) [0x00ADB49E]
	(No symbol) [0x00AC1CA6]
	(No symbol) [0x00A931D5]
	(No symbol) [0x00A9435D]
	GetHandleVerifier [0x00EF07C3+3142947]
	GetHandleVerifier [0x00F01A2B+3213195]
	GetHandleVerifier [0x00EFC412+3191154]
	GetHandleVerifier [0x00C98720+685184]
	(No symbol) [0x00B86E1D]
	(No symbol) [0x00B83E18]
	(No symbol) [0x00B83FB6]
	(No symbol) [0x00B766F0]
	BaseThreadInitThunk [0x7707FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7782809E+286]
	RtlGetAppContainerNamedObjectPath [0x7782806E+238]

Scraping project: http

KeyboardInterrupt: 

In [ ]:
import csv
import os
import requests
import time
import re
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from tempfile import NamedTemporaryFile 

service = Service(ChromeDriverManager().install())  # 自动安装对应版本的chromedriver
service.start()

# 设置 ChromeOptions
options = Options()
options.add_argument('--lang=en')  # 设置语言为英语

# 初始化 WebDriver
driver = WebDriver(service=service, options=options)
actions = ActionChains(driver)

def project_scraper(driver, project_link):
    try:
        response = requests.get(project_link)
        response.raise_for_status()  # 如果请求失败抛出异常
    except requests.exceptions.RequestException as e:
        print(f"Network error while fetching {project_link}: {e}")
        return "error"

    try:
        soup = BeautifulSoup(response.content, 'html.parser')
        print('Project details are being scraped.')

        # 初始化数据容器
        data = {
            'categories': [],
            'city': None,
            'country': None,
            'architects': [],
            'area': None,
            'year': None,
            'description': None  # 添加description字段
        }

        # 提取分类信息
        category_container = soup.find(class_='afd-specs__header-category')
        if category_container:
            data['categories'] = [a.text.strip() for a in category_container.find_all('a')]

        # 提取地理位置信息
        location_container = soup.find(class_='afd-specs__header-location')
        if location_container:
            location_parts = location_container.get_text(strip=True).split(',')
            if location_parts:
                data['city'] = location_parts[0].strip()
            country_link = location_container.find('a')
            if country_link:
                data['country'] = country_link.text.strip()

        # 遍历规格条目
        for item in soup.select('.afd-specs__item'):
            # 建筑师信息
            if 'Architects:' in item.text:
                architects = item.find(class_='afd-specs__value')
                if architects:
                    data['architects'] = [a.text.strip() for a in architects.find_all('a')]
                continue

            # 面积信息
            if 'Area:' in item.text:
                area_value = item.find(class_='afd-specs__value')
                if area_value:
                    data['area'] = area_value.text.replace('m²', '').strip()
                continue

            # 年份信息
            if 'Year:' in item.text:
                year_value = item.find(class_='afd-specs__value')
                if year_value:
                    data['year'] = year_value.text.strip()
                continue

        # 提取所有<p>标签的内容，并将其合并为description
        paragraphs = soup.find_all('p')
        description= []

        for paragraph in paragraphs:
            # Recursive function to extract text with proper spacing
            
            # Add text with preserved spacing
            extracted_text = extract_text_with_spacing(paragraph)
            # Normalize by removing excessive newlines and multiple spaces
            normalized_text = re.sub(r'\s+', ' ', extracted_text).strip()
            description.append(normalized_text)

        
        description = purge_description(description)
        if description:
            data['description'] = description

        # 构建最终数据结构
        project_dict = {
            'Project ID': project_link.split('/')[-1],
            'Project Title': soup.find(class_='afd-title-big--bmargin-big').get_text(strip=True).split('/')[0],
            'Categories': data['categories'] or None,
            'City': data['city'],
            'Country': data['country'],
            'Architects': data['architects'] or None,
            'Area': f"{data['area']} m²" if data['area'] else None,
            'Year': data['year'],
            'Project URL': project_link,
            'Description': data['description']  # 添加description字段
        }

        # 清理空值
        project_dict = {k: v for k, v in project_dict.items() if v is not None}

        # 写入JSON文件
        project_code = project_link.split('/')[-1]
        with open(f'{project_code}.json', 'w', encoding='utf-8') as f:
            json.dump(project_dict, f, ensure_ascii=False, indent=2)

        print('Project details are ready.')
        return "downloaded"

    except Exception as e:
        print(f"Error while processing {project_link}: {e}")
        return "error"


def extract_text_with_spacing(element):
    result = ""
    for content in element.contents:
        if content.name:  # if it's a tag
            result += extract_text_with_spacing(content)
        else:  # if it's NavigableString (text)
            result += content
    return result

def purge_description(description_list):
    # 删除所有按照空格分词后长度小于等于3的列表项,示例 ['Architects:', 'Architects Architects Architects Architects architects architects']应当被删得只剩下['Architects Architects Architects Architects architects architects']
    description_list = [x for x in description_list if len(x.split()) > 3]
    # 删除"You'll now receive updates based on what you follow! Personalize your stream and start following your favorite authors, offices and users."
    description_list = [x for x in description_list if x != "You'll now receive updates based on what you follow! Personalize your stream and start following your favorite authors, offices and users."]
    # 删除开头为”Check the“的项
    description_list = [x for x in description_list if not x.startswith("Check the")]
    # 删除重复的项
    description_list = list(dict.fromkeys(description_list))
    # 删除包含有"Save this picture!" 的项
    description_list = [x for x in description_list if "Save this picture!" not in x]

    return description_list



def project_image_downloader(driver, project_link):
    try:
        response = requests.get(project_link)
        soup = BeautifulSoup(response.content, 'html.parser')
        a_tag = soup.find('a', attrs={'class': 'gallery-link afd-desktop-e'})
        link = base_url + a_tag.get('href')
        driver.get(link)

        timeout = 20
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "js-gal-mob-img-onview", " " ))]')))

        this_table = driver.find_elements(By.ID, 'gallery-items')
        for this in this_table:
            temp = this.get_attribute('data-images')

        large_urls = []
        urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', temp)
        for url in urls:
            if 'large_jpg' in str(url):
                large_urls.append(url)
        
        i = 1
        project_code = project_link.split('/')[3]

        if not os.path.exists(project_code):
            os.makedirs(project_code)
        
        for each in large_urls:
            with open(os.path.join(project_code, f'{i}.jpg'), 'wb') as handle:
                response = requests.get(each, stream=True)
                print(f'Picture number {i} downloaded')
                time.sleep(2)

                if not response.ok:
                    print(response)

                for block in response.iter_content(1024):
                    if not block:
                        break

                    handle.write(block)
            i += 1
    except Exception as e:
        print(f"Error downloading images for {project_link}: {e}")

def update_csv_status(csv_file, project_code, status):
    # Read CSV and update status column
    rows = []
    with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        header = next(reader)
        status_index = header.index('status')  # 获取status列的索引
        rows.append(header)
        for row in reader:
            if row[0] == project_code:
                row[status_index] = status  # Update the status
            rows.append(row)

    # Write the updated content back to CSV
    with open(csv_file, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(rows)


def remove_csv_status(csv_file):
    # remove all the status in the csv file
    rows = []
    with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        header = next(reader)
        status_index = header.index('status')
        rows.append(header)

        for row in reader:
            row[status_index] = ""
            rows.append(row)s
    with open(csv_file, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(rows)

if __name__ == "__main__":
    csv_file = './archdaily_projects.csv'
    base_url = 'https://www.archdaily.com/'
    
    with open(csv_file, 'r', encoding='utf-8') as file:

        reader = csv.reader(file)

        header = next(reader)
        status_index = header.index('status')
        
        rows = list(reader)

        for row in rows:
            project_code = row[0]
            status = row[status_index]
            
            # Skip projects with status "downloaded", "error", or "incomplete"
            if status in ['downloaded', 'error', 'incomplete', 'duplicate', 'delete']:
                print(f"Skipping project {project_code} as it is marked as {status}.")
                continue
            
            project_link = base_url + project_code
            print(f"Scraping project: {project_link}")
            status = project_scraper(driver, project_link)
            

            update_csv_status(csv_file, project_code, status)
            remove_csv_status(csv_file)

            
            # # Only download images if status is not "incomplete"
            # if status != "incomplete" and status != "error":
            #     project_image_downloader(driver, project_link)


Scraping project: https://www.archdaily.com/904658
Project details are being scraped.
Project details are ready.
Scraping project: https://www.archdaily.com/965203
Project details are being scraped.
Project details are ready.
Scraping project: https://www.archdaily.com/343719
Project details are being scraped.
Project details are ready.
Scraping project: https://www.archdaily.com/563604
Project details are being scraped.
Project details are ready.
Scraping project: https://www.archdaily.com/883444
Project details are being scraped.
Project details are ready.
Scraping project: https://www.archdaily.com/778213
Project details are being scraped.
Project details are ready.
Scraping project: https://www.archdaily.com/339751
Project details are being scraped.
Project details are ready.
Scraping project: https://www.archdaily.com/801031
Project details are being scraped.
Project details are ready.


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

# 目标网页
url = "https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects"

# 发送请求获取页面内容
headers = {
    "User-Agent": "Mozilla/5.0"
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# 找到缩略图列表
thumb_list = soup.find("ul", class_="gallery-thumbs")

# 初始化结果列表
gallery_items = []

# 遍历每个 <li> 中的 <a> 标签
for li in thumb_list.find_all("li", class_="gallery-thumbs-item"):
    link = li.find("a", class_="gallery-thumbs-link")
    if link:
        href = urljoin(url, link.get("href", ""))
        title = link.get("title", "").strip()

        # 可扩展的数据结构
        item = {
            "href": href,
            "title": title,
            "tags": []  # 未来可以放标签信息
        }

        gallery_items.append(item)

# 新建一个文件夹用于保存图片，文件夹名为项目的ID，即url的除了archdaily.com/之后的数字id
project_id = url.split("/")[-1]


os.makedirs(project_id, exist_ok=True)

# 遍历每个缩略图链接，获取大图，下载到新建的文件夹中

for item in gallery_items:
    print(item)
    # 进入herf链接，将将"caption":""添加到item中，并在item的tags中添加所有的标签，最后下载图片，图片命名为 title_caption.jpg，title和caption都从item中获取




{'href': 'https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo', 'title': 'Phillip and Patricia Frost Museum of Science / Grimshaw Architects', 'tags': []}
{'href': 'https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1abe6f197ccd653000129-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo', 'title': 'Phillip and Patricia Frost Museum of Science / Grimshaw Architects', 'tags': []}
{'href': 'https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1abdbf197ccd653000128-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo', 'title': 'Phillip and Patricia Frost Museum of Science / Grimshaw Architects', 'tags': []}
{'href': 'https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5

In [6]:
def download_gallery_image(url, folder_name, image_name):
    """
    Download an image from an Archdaily gallery page and extract tags and caption
    
    Args:
        url (str): URL of the gallery page
        folder_name (str): Name of the folder to save the image
        image_name (str): Name to use for the saved image
    
    Returns:
        tuple: (bool, list, str) - Boolean success indicator, list of tags, and caption string
    """
    # Initialize empty tags list and caption string
    tags = []
    caption = ""
    
    # Request the page content
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch page: {response.status_code}")
        return False, tags, caption
    
    # Parse HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract caption from the figcaption element
    figcaption = soup.find('p', class_='afd-gal-figcaption')
    if figcaption:
        # If figcaption contains a link
        caption_link = figcaption.find('a')
        if caption_link:
            caption = caption_link.get_text(strip=True)
        else:
            # If it's just text
            caption = figcaption.get_text(strip=True)
    
    # Find the gallery-items div
    gallery_items = soup.find('div', id='gallery-items')
    if not gallery_items:
        print("Could not find gallery items div")
        return False, tags, caption
    
    # Extract the data-images attribute
    data_images_str = gallery_items.get('data-images')
    if not data_images_str:
        print("No data-images attribute found")
        return False, tags, caption
    
    try:
        # Clean the data-images string for proper JSON parsing
        data_images_str = data_images_str.replace('&quot;', '"')
        
        # If it's a partial string, we need to extract the JSON part
        if not data_images_str.startswith('['):
            # Use regex to find the JSON array
            match = re.search(r'\[(.*?)\]', data_images_str)
            if match:
                data_images_str = f"[{match.group(1)}]"
        
        # Parse the JSON data
        data_images = json.loads(data_images_str)
        
        # Get the url_large and tags from the first item
        if data_images and len(data_images) > 0:
            first_image = data_images[0]
            url_large = first_image.get('url_large')
            
            # Extract tags from the JSON data
            if 'tags' in first_image and isinstance(first_image['tags'], list):
                for tag_item in first_image['tags']:
                    if 'name' in tag_item and tag_item['name']:
                        tag_name = tag_item['name'].strip()
                        tags.append(tag_name)
            
            # If we didn't find caption in HTML, try getting it from JSON
            if not caption and 'caption' in first_image and first_image['caption']:
                caption = first_image['caption'].strip()
            
            if not url_large:
                print("No url_large found in the first item")
                return False, tags, caption
            
            # Get file extension from URL
            file_extension = os.path.splitext(url_large.split('/')[-1].split('?')[0])[1]
            if not file_extension:
                file_extension = ".jpg"  # Default extension if none found
                
            # Create full filename with extension
            full_image_name = f"{image_name}{file_extension}"
            
            # Download the image
            img_response = requests.get(url_large)
            if img_response.status_code == 200:
                # Create directory if it doesn't exist
                os.makedirs(folder_name, exist_ok=True)
                
                # Save the image
                with open(f"{folder_name}/{full_image_name}", 'wb') as file:
                    file.write(img_response.content)
                    
                # Print success message with tags and caption
                print(f"Image downloaded successfully: {folder_name}/{full_image_name}")
                if tags:
                    print(f"Image tags: {', '.join(tags)}")
                if caption:
                    print(f"Image caption: {caption}")
                
                return True, tags, caption
            else:
                print(f"Failed to download image: {img_response.status_code}")
                return False, tags, caption
        else:
            print("No items found in data_images")
            return False, tags, caption
            
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")
        print(f"Raw data: {data_images_str[:100]}...")  # Print first 100 chars for debugging
        return False, tags, caption
    except Exception as e:
        print(f"An error occurred: {e}")
        return False, tags, caption

# Example usage:
success, tags, caption = download_gallery_image(
    "https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1ab37f197ccd653000127-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-wind-diagram",
    "test",
    "main_view"
)


Image downloaded successfully: test/main_view.jpg
Image tags: Planetarium, Facade
Image caption: © Rafael Gamo


### 抓取图片链接

## 最终可用版本

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException # Added NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os
import time
import re

# --- Updated extract_gallery_tags function ---
def extract_gallery_tags(url, driver):
    """
    Extracts tags from an Archdaily gallery image page using Selenium
    to navigate and BeautifulSoup to parse the final page source.

    Args:
        url (str): The URL of the specific gallery image page.
        driver (WebDriver): The active Selenium WebDriver instance.

    Returns:
        list: List of tag strings found on the page.
    """
    tags = []
    print(f"  Navigating to image page for tags: {url}")
    try:
        driver.get(url)

        # Wait specifically for the tags container to be present on the new page
        # Increased timeout slightly just in case
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "afd-gal__tags"))
        )
        print(f"    Tags container found on page: {url}")

        # Get the page source AFTER navigation and waiting
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        # Use BeautifulSoup to find the tags container and tags
        tags_div = soup.find("div", class_="afd-gal__tags")
        if tags_div:
            # Tags are usually within 'a' elements inside the container
            tag_elements = tags_div.find_all("a", class_="afd-gal__tag")
            if not tag_elements:
                 # Fallback: Sometimes they might be spans or other elements
                 tag_elements = tags_div.find_all(class_="afd-gal__tag")

            for tag_element in tag_elements:
                tag_text = tag_element.get_text(strip=True)
                if tag_text:
                    tags.append(tag_text)

            if tags:
                print(f"    Found tags using BeautifulSoup: {', '.join(tags)}")
            else:
                print(f"    Tags container found, but no tag elements within it on {url}")
        else:
            # This case should be rare if the WebDriverWait succeeded
            print(f"    Tags container (afd-gal__tags) not found in BeautifulSoup parse on {url}")

    except TimeoutException:
        print(f"    TimeoutException: Could not find tags container (afd-gal__tags) on {url}")
    except Exception as e:
        # Catch other potential errors during navigation or parsing
        print(f"    Error processing tags for {url}: {str(e)}")

    if not tags:
        print(f"    No tags extracted for {url}")

    return tags

# --- scrape_gallery_thumbnails function (mostly unchanged, manages driver) ---
def scrape_gallery_thumbnails(url):
    """
    Scrapes gallery thumbnail information from an Archdaily project page.

    Args:
        url (str): The URL of the Archdaily project page

    Returns:
        tuple: (bool, list) - Success status and list of gallery items
    """
    gallery_items = []
    driver = None # Initialize driver to None

    try:
        # Set up the WebDriver for reuse across all requests
        print("Initializing WebDriver...")
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        # Using a common user agent
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        # Suppress logs slightly
        options.add_experimental_option('excludeSwitches', ['enable-logging'])

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("WebDriver Initialized.")

        # Send request with Selenium
        print(f"Fetching project page: {url}")
        driver.get(url)

        # Wait for the main gallery thumbnail section to load
        try:
            WebDriverWait(driver, 20).until( # Increased timeout for initial page load
                EC.presence_of_element_located((By.CLASS_NAME, "gallery-thumbs"))
            )
            print("Gallery thumbnails section loaded.")
        except TimeoutException:
            print("Timeout waiting for gallery thumbnails section (gallery-thumbs) to load on main page.")
            # Attempt to get source anyway, maybe it partially loaded
            pass # Continue and let BeautifulSoup try

        # Parse initial page HTML content with BeautifulSoup
        # Important: Get source *after* waiting for the element
        initial_page_source = driver.page_source
        soup = BeautifulSoup(initial_page_source, "html.parser")

        # Find thumbnail list
        thumb_list = soup.find("ul", class_="gallery-thumbs")
        if not thumb_list:
            print("Could not find thumbnail gallery (missing 'gallery-thumbs' ul element in parsed HTML)")
            # Check for alternative structure if needed, or return failure
            # Example: Look for individual items directly if ul is missing but items exist
            thumbnail_items_check = soup.find_all("li", class_="gallery-thumbs-item")
            if not thumbnail_items_check:
                 print("Also could not find individual 'gallery-thumbs-item' list items.")
                 return False, gallery_items
            else:
                 print("Warning: Found 'gallery-thumbs-item' but not the parent 'ul'. Processing items directly.")
                 # If items found directly, proceed without the thumb_list variable

        # Process each thumbnail
        success = False # Will be set to True if at least one item is processed
        print(f"Processing thumbnail items...")

        # Find all items relative to the main soup object if list wasn't found, or within the list
        thumbnail_items = soup.find_all("li", class_="gallery-thumbs-item")

        print(f"Found {len(thumbnail_items)} potential thumbnail items to process.")

        for i, li in enumerate(thumbnail_items):
            print(f"\nProcessing thumbnail item {i+1}/{len(thumbnail_items)}")
            try:
                link = li.find("a", class_="gallery-thumbs-link")
                if not link:
                    print("  Skipping item: No 'gallery-thumbs-link' found inside.")
                    continue

                # Resolve relative URLs correctly using urljoin
                href = link.get("href")
                if not href:
                    print("  Skipping item: Link found, but no 'href' attribute.")
                    continue
                full_href = urljoin(url, href) # Use urljoin!

                title = link.get("title", "").strip()
                if not title:
                    # Try to get title from img alt text as fallback
                    img = link.find("img")
                    if img:
                        title = img.get("alt", "").strip()
                print(f"  Found Link: {title} -> {full_href}")


                # Skip items with invalid or self-referencing URLs more robustly
                if not full_href or full_href == url or '#' in full_href.split('/')[-1]:
                    print(f"  Skipping item: Invalid or self-referencing URL ({full_href})")
                    continue

                # Create item data structure
                item = {
                    "href": full_href,
                    "title": title if title else "Untitled", # Provide default title
                    "tags": []
                }

                # Extract tags for this item using the shared WebDriver
                # Pass the absolute URL and the driver instance
                tags = extract_gallery_tags(full_href, driver) # Pass driver here
                if tags:
                    item["tags"] = tags

                gallery_items.append(item)
                success = True # At least one item was processed successfully

                # Small delay to avoid overloading the server
                time.sleep(0.5) # Keep the small delay

            except Exception as e:
                print(f"  Error processing thumbnail item {i+1}: {str(e)}")
                # Continue to next item instead of failing completely
                continue

        # Report results
        print(f"\nGallery scanning complete. Successfully processed {len(gallery_items)} items.")
        return success, gallery_items

    except Exception as e:
        print(f"An unexpected error occurred in scrape_gallery_thumbnails: {str(e)}")
        import traceback
        traceback.print_exc() # Print detailed traceback for debugging
        return False, gallery_items

    finally:
        # Clean up the WebDriver - IMPORTANT: This should always run
        if driver:
            print("Quitting WebDriver...")
            try:
                driver.quit()
                print("WebDriver quit successfully.")
            except Exception as e:
                print(f"Error quitting WebDriver: {e}")


# Example usage
if __name__ == "__main__":
    # Make sure to use a URL that actually has a gallery
    test_url = "https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects"

    start_time = time.time()
    success_status, scraped_items = scrape_gallery_thumbnails(test_url)
    end_time = time.time()

    print(f"\n--- Scraping Report ---")
    print(f"URL: {test_url}")
    print(f"Status: {'Success' if success_status else 'Failed'}")
    print(f"Items Scraped: {len(scraped_items)}")
    print(f"Total Time: {end_time - start_time:.2f} seconds")

    if success_status and scraped_items:
        print("\n--- Scraped Items ---")
        for i, item_data in enumerate(scraped_items, 1):
            print(f"{i}. Title: {item_data['title']}")
            print(f"   URL: {item_data['href']}")
            if item_data['tags']:
                print(f"   Tags: {', '.join(item_data['tags'])}")
            else:
                print(f"   Tags: [No tags found]")
            print("-" * 20)
    elif not success_status:
        print("\nScraping failed. Check logs for errors.")
    else:
        print("\nScraping finished but no items were successfully processed.")

Initializing WebDriver...
WebDriver Initialized.
Fetching project page: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects
Gallery thumbnails section loaded.
Processing thumbnail items...
Found 24 potential thumbnail items to process.

Processing thumbnail item 1/24
  Found Link: Phillip and Patricia Frost Museum of Science / Grimshaw Architects -> https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo
  Navigating to image page for tags: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo
    Tags container found on page: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia

In [11]:
import time
import requests
import json
import os
import re # For sanitizing filenames
from urllib.parse import urlparse, unquote, urljoin
from pathlib import Path
from bs4 import BeautifulSoup
import logging

# --- Selenium Imports ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import traceback # For detailed error printing

# Configure logging for download function (can be shared)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Helper Function for Sanitizing ---
def sanitize_foldername(name: str) -> str:
    """Removes potentially invalid characters for directory names."""
    # Remove characters not typically allowed in folder names
    name = re.sub(r'[<>:"/\\|?*]', '', name)
    # Replace spaces with underscores
    name = name.replace(' ', '_')
    # Truncate if too long (optional, e.g., limit to 50 chars)
    return name[:50].strip('_')

# --- scrape_gallery_thumbnails function (MODIFIED: removed extract_gallery_tags call) ---
def scrape_gallery_thumbnails(url):
    """
    Scrapes gallery thumbnail information (URL and title) from an Archdaily project page.

    Args:
        url (str): The URL of the Archdaily project page

    Returns:
        tuple: (bool, list) - Success status and list of gallery items [{'href': str, 'title': str}]
    """
    gallery_items = []
    driver = None # Initialize driver to None

    try:
        # Set up the WebDriver
        print("Initializing WebDriver...")
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        options.add_experimental_option('excludeSwitches', ['enable-logging'])

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("WebDriver Initialized.")

        # Send request with Selenium
        print(f"Fetching project page: {url}")
        driver.get(url)

        # Wait for the main gallery thumbnail section to load
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "gallery-thumbs"))
            )
            print("Gallery thumbnails section loaded.")
        except TimeoutException:
            print("Timeout waiting for gallery thumbnails section (gallery-thumbs) to load.")
            # Try to proceed anyway
            pass

        # Parse initial page HTML content
        initial_page_source = driver.page_source
        soup = BeautifulSoup(initial_page_source, "html.parser")

        # Find thumbnail list or individual items
        thumbnail_items = soup.find_all("li", class_="gallery-thumbs-item")
        if not thumbnail_items:
             print("Could not find any 'gallery-thumbs-item' list items.")
             return False, gallery_items

        print(f"Found {len(thumbnail_items)} potential thumbnail items to process.")
        success = False

        for i, li in enumerate(thumbnail_items):
            print(f"\nProcessing thumbnail item {i+1}/{len(thumbnail_items)}")
            try:
                link = li.find("a", class_="gallery-thumbs-link")
                if not link:
                    print("  Skipping item: No 'gallery-thumbs-link' found inside.")
                    continue

                href = link.get("href")
                if not href:
                    print("  Skipping item: Link found, but no 'href' attribute.")
                    continue

                # Resolve relative URLs correctly
                full_href = urljoin(url, href)

                title = link.get("title", "").strip()
                if not title:
                    img = link.find("img")
                    if img:
                        title = img.get("alt", "").strip()
                title = title if title else f"Untitled_{i+1}" # Provide default title

                print(f"  Found Link: {title} -> {full_href}")

                # Skip items with invalid or self-referencing URLs more robustly
                # (Checking fragment might be too strict here, rely on download function's logic)
                if not full_href or full_href == url:
                    print(f"  Skipping item: Invalid or self-referencing URL ({full_href})")
                    continue

                # Create item data structure (simplified, no tags extracted here)
                item = {
                    "href": full_href,
                    "title": title,
                }
                gallery_items.append(item)
                success = True

            except Exception as e:
                print(f"  Error processing thumbnail item {i+1}: {str(e)}")
                continue # Continue to next item

        print(f"\nGallery thumbnail scraping complete. Successfully processed {len(gallery_items)} items.")
        return success, gallery_items

    except Exception as e:
        print(f"An unexpected error occurred in scrape_gallery_thumbnails: {str(e)}")
        traceback.print_exc()
        return False, gallery_items

    finally:
        # Clean up the WebDriver
        if driver:
            print("Quitting WebDriver...")
            try:
                driver.quit()
                print("WebDriver quit successfully.")
            except Exception as e:
                print(f"Error quitting WebDriver: {e}")

# --- download_gallery_image function (mostly unchanged) ---
def download_gallery_image(page_url: str, base_folder: str, subfolder: str):
    """
    Downloads the currently displayed image from an ArchDaily gallery page.

    It identifies the target image based on the URL fragment (#...) if present,
    otherwise uses the default image indicated by 'data-id' on the gallery element.

    Args:
        page_url: The full URL of the ArchDaily gallery page, potentially
                  including a fragment identifier (#...) pointing to a specific image.
        base_folder: The main directory where the subfolder will be created.
        subfolder: The name of the subfolder within the base_folder to save the image.

    Returns:
        A tuple containing:
        - success (bool): True if download and parsing were successful, False otherwise.
        - tags (list[str] | None): A list of tag names for the image, or None on failure.
        - caption (str | None): The caption text for the image, or None on failure.
    """
    success = False
    tags = None
    caption = None
    target_image_info = None

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    } # Mimic a browser to avoid potential blocking

    try:
        # 1. Fetch HTML content
        logging.info(f"Fetching HTML from: {page_url}")
        response = requests.get(page_url, headers=headers, timeout=15)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        logging.info("HTML fetched successfully.")

        # 2. Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # 3. Find the gallery data container and extract JSON
        gallery_div = soup.find('div', id='gallery-items')
        if not gallery_div:
            logging.error("Could not find the gallery container ('div#gallery-items').")
            return success, tags, caption

        data_images_json = gallery_div.get('data-images')
        default_image_id_fragment = gallery_div.get('data-id') # Get default ID for fallback

        if not data_images_json:
            logging.error("Could not find 'data-images' attribute in the gallery container.")
            return success, tags, caption

        # 4. Parse JSON data
        try:
            image_data = json.loads(data_images_json)
            if not isinstance(image_data, list) or not image_data:
                 logging.error("'data-images' did not contain a valid list of images.")
                 return success, tags, caption
            logging.info(f"Successfully parsed data for {len(image_data)} images.")
        except json.JSONDecodeError as e:
            logging.error(f"Failed to parse JSON from 'data-images': {e}")
            return success, tags, caption

        # 5. Identify the target image
        parsed_url = urlparse(page_url)
        url_fragment = parsed_url.fragment

        if url_fragment:
            logging.info(f"URL fragment detected: '{url_fragment}'. Searching for matching image link.")
            normalized_fragment = url_fragment.split('-')[0]
            found_by_fragment = False
            for img_info in image_data:
                if 'link' in img_info and url_fragment in img_info['link']:
                     target_image_info = img_info
                     logging.info(f"Found image matching fragment: ID {target_image_info.get('link', 'N/A')}")
                     found_by_fragment = True
                     break
                elif 'link' in img_info and img_info['link'].startswith(normalized_fragment):
                     target_image_info = img_info
                     logging.info(f"Found image matching normalized fragment start: ID {target_image_info.get('link', 'N/A')}")
                     found_by_fragment = True
                     break
            if not found_by_fragment:
                 logging.warning(f"Could not find image matching fragment '{url_fragment}'. Falling back to default.")

        if not target_image_info:
            if default_image_id_fragment:
                logging.info(f"Using default image ID from 'data-id': '{default_image_id_fragment}'. Searching...")
                found_by_default_id = False
                for img_info in image_data:
                     if 'link' in img_info and default_image_id_fragment in img_info['link']:
                         target_image_info = img_info
                         logging.info(f"Found default image: ID {target_image_info.get('link', 'N/A')}")
                         found_by_default_id = True
                         break
                if not found_by_default_id:
                     logging.warning(f"Could not find image matching default 'data-id' {default_image_id_fragment}. Using first image.")
                     target_image_info = image_data[0]
            else:
                logging.warning("No URL fragment and no 'data-id' found. Using first image.")
                target_image_info = image_data[0]

        if not target_image_info:
             logging.error("Could not determine target image.")
             return success, tags, caption

        # 6. Extract details from the target image info
        image_url = target_image_info.get('url_large') or target_image_info.get('url_slideshow')
        caption = target_image_info.get('caption', 'No Caption')
        raw_tags = target_image_info.get('tags', [])
        tags = [tag.get('name', '').strip() for tag in raw_tags if isinstance(tag, dict) and 'name' in tag]

        if not image_url:
            logging.error(f"Target image object (ID: {target_image_info.get('link')}) is missing 'url_large' or 'url_slideshow'.")
            return success, tags, caption

        logging.info(f"Target Image URL: {image_url}")
        logging.info(f"Caption: {caption}")
        logging.info(f"Tags: {tags}")

        # 7. Prepare download path and filename
        # Ensure subfolder is sanitized before creating the path
        safe_subfolder = sanitize_foldername(subfolder)
        save_dir = Path(base_folder) / safe_subfolder
        save_dir.mkdir(parents=True, exist_ok=True)

        parsed_image_url = urlparse(image_url)
        image_filename = Path(unquote(parsed_image_url.path)).name

        if not image_filename:
             image_filename = target_image_info.get('link', 'downloaded_image').split('/')[-1] + ".jpg"
             image_filename = sanitize_foldername(image_filename) # Sanitize fallback too

        save_path = save_dir / image_filename
        logging.info(f"Image will be saved to: {save_path}")

        # 8. Download and save the image
        logging.info(f"Downloading image from {image_url}...")
        img_response = requests.get(image_url, stream=True, headers=headers, timeout=30)
        img_response.raise_for_status()

        with open(save_path, 'wb') as f:
            for chunk in img_response.iter_content(chunk_size=8192):
                f.write(chunk)

        logging.info(f"Image successfully downloaded and saved to {save_path}")
        success = True

    except requests.exceptions.Timeout as e:
        logging.error(f"Request timed out: {e}")
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP Error fetching page or image: {e.response.status_code} - {e}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error: {e}")
    except FileNotFoundError:
         logging.error(f"Error creating directory or file at {save_path}. Check permissions or base path.")
    except Exception as e:
        logging.error(f"An unexpected error occurred in download_gallery_image: {e}", exc_info=True)

    finally:
        return success, tags, caption


# --- Main Execution Logic ---
if __name__ == "__main__":
    PROJECT_URL = "https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects"
    BASE_DOWNLOAD_FOLDER = "archdaily_frost_museum" # Base folder for this project's downloads

    print(f"--- Starting scraping process for: {PROJECT_URL} ---")

    # 1. Scrape all thumbnail URLs and titles
    success_scrape, gallery_items = scrape_gallery_thumbnails(PROJECT_URL)

    if not success_scrape or not gallery_items:
        print("\n--- Thumbnail scraping failed or no items found. Exiting. ---")
    else:
        print(f"\n--- Starting download process for {len(gallery_items)} items ---")
        print(f"Downloads will be saved under '{BASE_DOWNLOAD_FOLDER}'")

        # 2. Iterate and download each image
        for index, item in enumerate(gallery_items):
            item_url = item.get('href')
            item_title = item.get('title', f'image_{index+1}')

            print(f"\n>>> Processing Item {index+1}/{len(gallery_items)}: '{item_title}' <<<")
            print(f"    URL: {item_url}")

            if not item_url:
                print("    Skipping item: Missing URL.")
                continue

            # Create a subfolder name based on index and title
            subfolder_name = f"{index+1:02d}_{sanitize_foldername(item_title)}"
            print(f"    Target Subfolder: {subfolder_name}")


            # Call the download function
            success_download, tags, caption = download_gallery_image(
                item_url,
                BASE_DOWNLOAD_FOLDER,
                subfolder_name
            )

            # Report download result
            if success_download:
                print(f"    Download Status: SUCCESS")
                print(f"      Caption: {caption}")
                print(f"      Tags: {tags}")
            else:
                print(f"    Download Status: FAILED")
                # Log tags/caption even if download failed, if they were parsed
                if caption or tags:
                     print(f"      Caption (if parsed): {caption}")
                     print(f"      Tags (if parsed): {tags}")

            # Add a delay between requests to be polite
            print("    Waiting 1 second before next item...")
            time.sleep(1) # Pause for 1 second

        print("\n--- All items processed. ---")

2025-04-06 19:53:16,734 - INFO - ====== WebDriver manager ======


--- Starting scraping process for: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects ---
Initializing WebDriver...


2025-04-06 19:53:17,543 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-06 19:53:18,966 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-06 19:53:20,207 - INFO - Driver [C:\Users\15717\.wdm\drivers\chromedriver\win64\134.0.6998.165\chromedriver-win32/chromedriver.exe] found in cache


WebDriver Initialized.
Fetching project page: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects
Gallery thumbnails section loaded.
Found 24 potential thumbnail items to process.

Processing thumbnail item 1/24
  Found Link: Phillip and Patricia Frost Museum of Science / Grimshaw Architects -> https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo

Processing thumbnail item 2/24
  Found Link: Phillip and Patricia Frost Museum of Science / Grimshaw Architects -> https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1abe6f197ccd653000129-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo

Processing thumbnail item 3/24
  Found Link: Phillip and Patricia Frost Museum of Science / Grimshaw Architects -> https://www.archdaily.com/904658/phill

2025-04-06 19:53:54,729 - INFO - Fetching HTML from: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo


...

    Download Status: SUCCESS
      Caption: Wind Diagram
      Tags: ['Drawings', 'Planetarium']
    Waiting 1 second before next item...


2025-04-06 19:55:56,872 - INFO - Fetching HTML from: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1ab2cf197cc96d0000331-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-water-diagram



>>> Processing Item 24/24: 'Phillip and Patricia Frost Museum of Science / Grimshaw Architects' <<<
    URL: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1ab2cf197cc96d0000331-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-water-diagram
    Target Subfolder: 24_Phillip_and_Patricia_Frost_Museum_of_Science__Grim


2025-04-06 19:55:58,917 - INFO - HTML fetched successfully.
2025-04-06 19:55:58,928 - INFO - Successfully parsed data for 24 images.
2025-04-06 19:55:58,929 - INFO - Using default image ID from 'data-id': '5bd1ab2cf197cc96d0000331'. Searching...
2025-04-06 19:55:58,929 - INFO - Found default image: ID 5bd1ab2cf197cc96d0000331-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-water-diagram
2025-04-06 19:55:58,930 - INFO - Target Image URL: https://images.adsttc.com/media/images/5bd1/ab2c/f197/cc96/d000/0331/large_jpg/03_CD_2008-1221_water.jpg?1540467493
2025-04-06 19:55:58,930 - INFO - Caption: Water Diagram
2025-04-06 19:55:58,931 - INFO - Tags: ['Drawings', 'Planetarium']
2025-04-06 19:55:58,932 - INFO - Image will be saved to: archdaily_frost_museum\24_Phillip_and_Patricia_Frost_Museum_of_Science__G\03_CD_2008-1221_water.jpg
2025-04-06 19:55:58,932 - INFO - Downloading image from https://images.adsttc.com/media/images/5bd1/ab2c/f197/cc96/d000/0331/large_jpg/03_CD_2008-

    Download Status: SUCCESS
      Caption: Water Diagram
      Tags: ['Drawings', 'Planetarium']
    Waiting 1 second before next item...

--- All items processed. ---


## 链接+爬图正式版


In [12]:
import time
import requests
import json
import os
import re # For sanitizing filenames
from urllib.parse import urlparse, unquote, urljoin
from pathlib import Path
from bs4 import BeautifulSoup
import logging

# --- Selenium Imports ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import traceback # For detailed error printing

# Configure logging (can be shared)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger("requests").setLevel(logging.WARNING) # Quieten requests library logging
logging.getLogger("urllib3").setLevel(logging.WARNING) # Quieten urllib3 library logging
logging.getLogger("selenium").setLevel(logging.WARNING) # Quieten selenium logging
logging.getLogger("webdriver_manager").setLevel(logging.WARNING) # Quieten webdriver-manager logging


# --- Helper Function (No longer needed for folder names, potentially for filenames later) ---
# def sanitize_filename(name: str) -> str:
#     """Removes potentially invalid characters for filenames."""
#     name = re.sub(r'[<>:"/\\|?*]', '', name)
#     return name[:100] # Limit length


# --- scrape_gallery_thumbnails function (Unchanged from previous version) ---
def scrape_gallery_thumbnails(url):
    """
    Scrapes gallery thumbnail information (URL and title) from an Archdaily project page.

    Args:
        url (str): The URL of the Archdaily project page

    Returns:
        tuple: (bool, list) - Success status and list of gallery items [{'href': str, 'title': str}]
    """
    gallery_items = []
    driver = None

    try:
        print("Initializing WebDriver...")
        options = Options()
        # Make headless quieter
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--log-level=3') # Suppress console logs from Chrome/ChromeDriver
        options.add_experimental_option('excludeSwitches', ['enable-logging']) # Suppress DevTools logging
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

        # Suppress webdriver-manager logs by configuring its logger
        os.environ['WDM_LOG_LEVEL'] = '0' # Set WDM log level to suppress output

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("WebDriver Initialized.")

        print(f"Fetching project page: {url}")
        driver.get(url)

        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "gallery-thumbs"))
            )
            print("Gallery thumbnails section loaded.")
        except TimeoutException:
            print("Timeout waiting for gallery thumbnails section (gallery-thumbs) to load.")
            pass # Try to proceed

        initial_page_source = driver.page_source
        soup = BeautifulSoup(initial_page_source, "html.parser")

        thumbnail_items = soup.find_all("li", class_="gallery-thumbs-item")
        if not thumbnail_items:
             print("Could not find any 'gallery-thumbs-item' list items.")
             return False, gallery_items

        print(f"Found {len(thumbnail_items)} potential thumbnail items to process.")
        success = False

        for i, li in enumerate(thumbnail_items):
            # Reduced print frequency inside the loop for cleaner output
            # print(f"\nProcessing thumbnail item {i+1}/{len(thumbnail_items)}")
            try:
                link = li.find("a", class_="gallery-thumbs-link")
                if not link: continue
                href = link.get("href")
                if not href: continue

                full_href = urljoin(url, href)
                title = link.get("title", "").strip()
                if not title:
                    img = link.find("img")
                    if img: title = img.get("alt", "").strip()
                title = title if title else f"Untitled_{i+1}"

                # print(f"  Found Link: {title} -> {full_href}") # Optional: uncomment for detailed link logging

                if not full_href or full_href == url: continue

                item = {"href": full_href, "title": title}
                gallery_items.append(item)
                success = True

            except Exception as e:
                print(f"  Error processing thumbnail item {i+1}: {str(e)}")
                continue

        print(f"\nGallery thumbnail scraping complete. Successfully extracted {len(gallery_items)} items.")
        return success, gallery_items

    except Exception as e:
        print(f"An unexpected error occurred in scrape_gallery_thumbnails: {str(e)}")
        traceback.print_exc()
        return False, gallery_items

    finally:
        if driver:
            print("Quitting WebDriver...")
            try:
                driver.quit()
                print("WebDriver quit successfully.")
            except Exception as e: print(f"Error quitting WebDriver: {e}")


# --- download_gallery_image function (MODIFIED SIGNATURE AND LOGIC) ---
def download_gallery_image(page_url: str, save_directory: Path, base_filename: str):
    """
    Downloads the specific image from an ArchDaily gallery page URL, determines its
    extension, saves it to the specified directory with the base filename + extension,
    and returns metadata including the final filename.

    Args:
        page_url: The full URL of the specific ArchDaily gallery image page.
        save_directory: The pathlib.Path object representing the directory to save the image.
        base_filename: The base name for the image file (e.g., "projectID_index").

    Returns:
        A tuple containing:
        - success (bool): True if download and parsing were successful, False otherwise.
        - tags (list[str] | None): A list of tag names for the image, or None on failure.
        - caption (str | None): The caption text for the image, or None on failure.
        - final_filename (str | None): The actual filename used for saving (including extension), or None.
    """
    success = False
    tags = None
    caption = None
    target_image_info = None
    final_filename = None # Initialize final filename

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        # 1. Fetch HTML content
        logging.info(f"Fetching HTML from: {page_url}")
        response = requests.get(page_url, headers=headers, timeout=15)
        response.raise_for_status()
        logging.info("HTML fetched successfully.")

        # 2. Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # 3. Find the gallery data container and extract JSON
        gallery_div = soup.find('div', id='gallery-items')
        if not gallery_div:
            logging.error("Could not find the gallery container ('div#gallery-items').")
            return success, tags, caption, final_filename

        data_images_json = gallery_div.get('data-images')
        default_image_id_fragment = gallery_div.get('data-id')

        if not data_images_json:
            logging.error("Could not find 'data-images' attribute.")
            return success, tags, caption, final_filename

        # 4. Parse JSON data
        try:
            image_data = json.loads(data_images_json)
            if not isinstance(image_data, list) or not image_data:
                 logging.error("'data-images' did not contain a valid list of images.")
                 return success, tags, caption, final_filename
            # logging.info(f"Successfully parsed data for {len(image_data)} images.") # Reduced verbosity
        except json.JSONDecodeError as e:
            logging.error(f"Failed to parse JSON from 'data-images': {e}")
            return success, tags, caption, final_filename

        # 5. Identify the target image (using fragment or default ID)
        parsed_url = urlparse(page_url)
        url_fragment = parsed_url.fragment

        if url_fragment:
            # Simplified fragment matching - look for the ID part primarily
            target_id = url_fragment.split('-')[0] # Extract potential ID
            # logging.info(f"URL fragment detected: '{url_fragment}'. Searching for link containing ID '{target_id}'.")
            found_by_fragment = False
            for img_info in image_data:
                if 'link' in img_info and target_id in img_info['link']:
                     target_image_info = img_info
                     # logging.info(f"Found image matching fragment ID: Link {target_image_info.get('link', 'N/A')}")
                     found_by_fragment = True
                     break
            if not found_by_fragment:
                 logging.warning(f"Could not find image matching fragment ID '{target_id}'. Falling back to default.")

        if not target_image_info: # Fallback to default image ID
            if default_image_id_fragment:
                # logging.info(f"Using default image ID from 'data-id': '{default_image_id_fragment}'. Searching...")
                found_by_default_id = False
                for img_info in image_data:
                     if 'link' in img_info and default_image_id_fragment in img_info['link']:
                         target_image_info = img_info
                         # logging.info(f"Found default image: Link {target_image_info.get('link', 'N/A')}")
                         found_by_default_id = True
                         break
                if not found_by_default_id:
                     logging.warning(f"Could not find image matching default 'data-id' {default_image_id_fragment}. Using first image.")
                     if image_data: target_image_info = image_data[0]
            else:
                logging.warning("No URL fragment and no 'data-id' found. Using first image.")
                if image_data: target_image_info = image_data[0]

        if not target_image_info:
             logging.error("Could not determine target image from parsed data.")
             return success, tags, caption, final_filename

        # 6. Extract details
        image_url = target_image_info.get('url_large') or target_image_info.get('url_slideshow')
        caption = target_image_info.get('caption', 'No Caption')
        raw_tags = target_image_info.get('tags', [])
        tags = [tag.get('name', '').strip() for tag in raw_tags if isinstance(tag, dict) and 'name' in tag]

        if not image_url:
            logging.error(f"Target image object (Link: {target_image_info.get('link')}) missing image URL.")
            return success, tags, caption, final_filename # Return parsed tags/caption

        # logging.info(f"Target Image URL: {image_url}") # Reduced verbosity
        # logging.info(f"Caption: {caption}")
        # logging.info(f"Tags: {tags}")

        # 7. Determine filename with extension
        parsed_image_url = urlparse(image_url)
        image_path_part = Path(unquote(parsed_image_url.path))
        extension = image_path_part.suffix # Get extension like .jpg, .png
        if not extension or len(extension) > 5: # Basic check for valid extension
             extension = ".jpg" # Default if no extension found or it's weirdly long
             logging.warning(f"Could not determine valid extension from {image_url}, defaulting to .jpg")

        final_filename = f"{base_filename}{extension}"
        save_path = save_directory / final_filename
        logging.info(f"Image will be saved as: {save_path}")

        # 8. Download and save the image
        logging.info(f"Downloading image from {image_url}...")
        img_response = requests.get(image_url, stream=True, headers=headers, timeout=30)
        img_response.raise_for_status()

        with open(save_path, 'wb') as f:
            for chunk in img_response.iter_content(chunk_size=8192):
                f.write(chunk)

        logging.info(f"Image successfully downloaded.")
        success = True

    except requests.exceptions.Timeout as e:
        logging.error(f"Request timed out: {e}")
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP Error fetching page or image: {e.response.status_code} - {e}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error: {e}")
    except FileNotFoundError:
         logging.error(f"Error creating directory or file at {save_path}. Check permissions or base path.")
    except Exception as e:
        logging.error(f"An unexpected error occurred in download_gallery_image: {e}", exc_info=False) # Set exc_info=False for less verbose logs on common errors

    finally:
        # Return status, metadata, AND the final filename used (or None)
        return success, tags, caption, final_filename


# --- Main Execution Logic ---
if __name__ == "__main__":
    PROJECT_URL = "https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects"

    # --- Extract Project ID and Set Up Base Folder ---
    project_id = None
    try:
        # Extract the first sequence of digits found in the path as the ID
        path_parts = [part for part in urlparse(PROJECT_URL).path.strip('/').split('/') if part]
        for part in path_parts:
            if part.isdigit():
                project_id = part
                break
    except Exception as e:
        print(f"Error extracting project ID from URL '{PROJECT_URL}': {e}")

    if not project_id:
        print(f"Could not extract numeric project ID from URL: {PROJECT_URL}. Exiting.")
        exit()

    BASE_DOWNLOAD_FOLDER = Path(project_id) # Base folder is named after the project ID
    print(f"Project ID detected: {project_id}")
    print(f"Base download directory set to: '{BASE_DOWNLOAD_FOLDER.resolve()}'")

    # Create the base directory
    try:
        BASE_DOWNLOAD_FOLDER.mkdir(parents=True, exist_ok=True)
        print(f"Ensured base directory exists.")
    except OSError as e:
        print(f"Error creating base directory '{BASE_DOWNLOAD_FOLDER}': {e}. Please check permissions. Exiting.")
        exit()

    # --- Start Scraping ---
    print(f"\n--- Starting scraping process for: {PROJECT_URL} ---")
    success_scrape, gallery_items = scrape_gallery_thumbnails(PROJECT_URL)

    if not success_scrape or not gallery_items:
        print("\n--- Thumbnail scraping failed or no items found. Exiting. ---")
    else:
        print(f"\n--- Starting download process for {len(gallery_items)} items ---")

        all_metadata = [] # Initialize list to store metadata for JSON

        # --- Iterate and Download ---
        for index, item in enumerate(gallery_items):
            item_url = item.get('href')
            item_title = item.get('title', f'image_{index+1}') # Use title for logging

            print(f"\n>>> Processing Item {index+1}/{len(gallery_items)}: '{item_title}' <<<")
            print(f"    Source URL: {item_url}")

            if not item_url:
                print("    Skipping item: Missing URL.")
                continue

            # Generate the base filename (ProjectID_Index)
            base_filename = f"{project_id}_{index+1:02d}"

            # Call the download function - Pass directory Path object and base filename
            success_download, tags, caption, final_filename = download_gallery_image(
                item_url,
                BASE_DOWNLOAD_FOLDER, # Pass the Path object for the directory
                base_filename         # Pass the base name without extension
            )

            # --- Report and Collect Metadata ---
            if success_download and final_filename:
                print(f"    Download Status: SUCCESS")
                print(f"      Saved as: {final_filename}")
                print(f"      Caption: {caption}")
                print(f"      Tags: {tags}")
                # Add metadata to the list
                metadata_entry = {
                    "filename": final_filename,
                    "tags": tags if tags else [], # Ensure tags is a list
                    "caption": caption if caption else "" # Ensure caption is a string
                }
                all_metadata.append(metadata_entry)
            else:
                print(f"    Download Status: FAILED")
                if final_filename: # Log filename even if download failed but naming succeeded
                     print(f"      Attempted filename: {final_filename}")
                # Log tags/caption if they were parsed even if download failed
                if caption or tags:
                     print(f"      Caption (if parsed): {caption}")
                     print(f"      Tags (if parsed): {tags}")

            # Add a delay between requests
            print("    Waiting 1 second before next item...")
            time.sleep(1)

        # --- Save Metadata JSON ---
        if all_metadata:
            metadata_filepath = BASE_DOWNLOAD_FOLDER / f"{project_id}.json"
            print(f"\n--- Saving metadata for {len(all_metadata)} items to {metadata_filepath} ---")
            try:
                with open(metadata_filepath, 'w', encoding='utf-8') as f:
                    json.dump(all_metadata, f, indent=4, ensure_ascii=False)
                print("Metadata successfully saved.")
            except IOError as e:
                print(f"Error saving metadata JSON file: {e}")
        else:
            print("\n--- No metadata collected, skipping JSON file creation. ---")

        print("\n--- All items processed. ---")

2025-04-06 20:03:19,887 - INFO - ====== WebDriver manager ======


Project ID detected: 904658
Base download directory set to: 'D:\999_RAG\crawler\904658'
Ensured base directory exists.

--- Starting scraping process for: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects ---
Initializing WebDriver...


2025-04-06 20:03:20,641 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-06 20:03:21,777 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-06 20:03:22,984 - INFO - Driver [C:\Users\15717\.wdm\drivers\chromedriver\win64\134.0.6998.165\chromedriver-win32/chromedriver.exe] found in cache


WebDriver Initialized.
Fetching project page: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects
Gallery thumbnails section loaded.
Found 24 potential thumbnail items to process.

Gallery thumbnail scraping complete. Successfully extracted 24 items.
Quitting WebDriver...
WebDriver quit successfully.


2025-04-06 20:03:54,678 - INFO - Fetching HTML from: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo


...

    Download Status: SUCCESS
      Saved as: 904658_23.jpg
      Caption: Wind Diagram
      Tags: ['Drawings', 'Planetarium']
    Waiting 1 second before next item...


2025-04-06 20:05:41,748 - INFO - Fetching HTML from: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1ab2cf197cc96d0000331-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-water-diagram



>>> Processing Item 24/24: 'Phillip and Patricia Frost Museum of Science / Grimshaw Architects' <<<
    Source URL: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1ab2cf197cc96d0000331-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-water-diagram


2025-04-06 20:05:43,438 - INFO - HTML fetched successfully.
2025-04-06 20:05:43,447 - INFO - Image will be saved as: 904658\904658_24.jpg
2025-04-06 20:05:43,448 - INFO - Downloading image from https://images.adsttc.com/media/images/5bd1/ab2c/f197/cc96/d000/0331/large_jpg/03_CD_2008-1221_water.jpg?1540467493...
2025-04-06 20:05:45,962 - INFO - Image successfully downloaded.


    Download Status: SUCCESS
      Saved as: 904658_24.jpg
      Caption: Water Diagram
      Tags: ['Drawings', 'Planetarium']
    Waiting 1 second before next item...

--- Saving metadata for 24 items to 904658\904658.json ---
Metadata successfully saved.

--- All items processed. ---


In [ ]:
import csv
import os
import requests
import time
import re
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from tempfile import NamedTemporaryFile 

service = Service(ChromeDriverManager().install())  # 自动安装对应版本的chromedriver
service.start()

# 设置 ChromeOptions
options = Options()
options.add_argument('--lang=en')  # 设置语言为英语

# 初始化 WebDriver
driver = WebDriver(service=service, options=options)
actions = ActionChains(driver)

def project_scraper(driver, project_link):
    try:
        response = requests.get(project_link)
        response.raise_for_status()  # 如果请求失败抛出异常
    except requests.exceptions.RequestException as e:
        print(f"Network error while fetching {project_link}: {e}")
        return "error"

    try:
        soup = BeautifulSoup(response.content, 'html.parser')
        print('Project details are being scraped.')

        # 初始化数据容器
        data = {
            'categories': [],
            'city': None,
            'country': None,
            'architects': [],
            'area': None,
            'year': None,
            'description': None  # 添加description字段
        }

        # 提取分类信息
        category_container = soup.find(class_='afd-specs__header-category')
        if category_container:
            data['categories'] = [a.text.strip() for a in category_container.find_all('a')]

        # 提取地理位置信息
        location_container = soup.find(class_='afd-specs__header-location')
        if location_container:
            location_parts = location_container.get_text(strip=True).split(',')
            if location_parts:
                data['city'] = location_parts[0].strip()
            country_link = location_container.find('a')
            if country_link:
                data['country'] = country_link.text.strip()

        # 遍历规格条目
        for item in soup.select('.afd-specs__item'):
            # 建筑师信息
            if 'Architects:' in item.text:
                architects = item.find(class_='afd-specs__value')
                if architects:
                    data['architects'] = [a.text.strip() for a in architects.find_all('a')]
                continue

            # 面积信息
            if 'Area:' in item.text:
                area_value = item.find(class_='afd-specs__value')
                if area_value:
                    data['area'] = area_value.text.replace('m²', '').strip()
                continue

            # 年份信息
            if 'Year:' in item.text:
                year_value = item.find(class_='afd-specs__value')
                if year_value:
                    data['year'] = year_value.text.strip()
                continue

        # 提取所有<p>标签的内容，并将其合并为description
        paragraphs = soup.find_all('p')
        description= []

        for paragraph in paragraphs:
            # Recursive function to extract text with proper spacing
            
            # Add text with preserved spacing
            extracted_text = extract_text_with_spacing(paragraph)
            # Normalize by removing excessive newlines and multiple spaces
            normalized_text = re.sub(r'\s+', ' ', extracted_text).strip()
            description.append(normalized_text)

        
        description = purge_description(description)
        if description:
            data['description'] = description

        # 构建最终数据结构
        project_dict = {
            'Project ID': project_link.split('/')[-1],
            'Project Title': soup.find(class_='afd-title-big--bmargin-big').get_text(strip=True).split('/')[0],
            'Categories': data['categories'] or None,
            'City': data['city'],
            'Country': data['country'],
            'Architects': data['architects'] or None,
            'Area': f"{data['area']} m²" if data['area'] else None,
            'Year': data['year'],
            'Project URL': project_link,
            'Description': data['description']  # 添加description字段
        }

        # 清理空值
        project_dict = {k: v for k, v in project_dict.items() if v is not None}

        # 写入JSON文件
        project_code = project_link.split('/')[-1]
        with open(f'{project_code}.json', 'w', encoding='utf-8') as f:
            json.dump(project_dict, f, ensure_ascii=False, indent=2)

        print('Project details are ready.')
        return "downloaded"

    except Exception as e:
        print(f"Error while processing {project_link}: {e}")
        return "error"


def extract_text_with_spacing(element):
    result = ""
    for content in element.contents:
        if content.name:  # if it's a tag
            result += extract_text_with_spacing(content)
        else:  # if it's NavigableString (text)
            result += content
    return result

def purge_description(description_list):
    # 删除所有按照空格分词后长度小于等于3的列表项,示例 ['Architects:', 'Architects Architects Architects Architects architects architects']应当被删得只剩下['Architects Architects Architects Architects architects architects']
    description_list = [x for x in description_list if len(x.split()) > 3]
    # 删除"You'll now receive updates based on what you follow! Personalize your stream and start following your favorite authors, offices and users."
    description_list = [x for x in description_list if x != "You'll now receive updates based on what you follow! Personalize your stream and start following your favorite authors, offices and users."]
    # 删除开头为”Check the“的项
    description_list = [x for x in description_list if not x.startswith("Check the")]
    # 删除重复的项
    description_list = list(dict.fromkeys(description_list))
    # 删除包含有"Save this picture!" 的项
    description_list = [x for x in description_list if "Save this picture!" not in x]

    return description_list



def project_image_downloader(driver, project_link):
    try:
        response = requests.get(project_link)
        soup = BeautifulSoup(response.content, 'html.parser')
        a_tag = soup.find('a', attrs={'class': 'gallery-link afd-desktop-e'})
        link = base_url + a_tag.get('href')
        driver.get(link)

        timeout = 20
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "js-gal-mob-img-onview", " " ))]')))

        this_table = driver.find_elements(By.ID, 'gallery-items')
        for this in this_table:
            temp = this.get_attribute('data-images')

        large_urls = []
        urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', temp)
        for url in urls:
            if 'large_jpg' in str(url):
                large_urls.append(url)
        
        i = 1
        project_code = project_link.split('/')[3]

        if not os.path.exists(project_code):
            os.makedirs(project_code)
        
        for each in large_urls:
            with open(os.path.join(project_code, f'{i}.jpg'), 'wb') as handle:
                response = requests.get(each, stream=True)
                print(f'Picture number {i} downloaded')
                time.sleep(2)

                if not response.ok:
                    print(response)

                for block in response.iter_content(1024):
                    if not block:
                        break

                    handle.write(block)
            i += 1
    except Exception as e:
        print(f"Error downloading images for {project_link}: {e}")

def update_csv_status(csv_file, project_code, status):
    # Read CSV and update status column
    rows = []
    with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        header = next(reader)
        status_index = header.index('status')  # 获取status列的索引
        rows.append(header)
        for row in reader:
            if row[0] == project_code:
                row[status_index] = status  # Update the status
            rows.append(row)

    # Write the updated content back to CSV
    with open(csv_file, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(rows)


def remove_csv_status(csv_file):
    # remove all the status in the csv file
    rows = []
    with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        header = next(reader)
        status_index = header.index('status')
        rows.append(header)

        for row in reader:
            row[status_index] = ""
            rows.append(row)s
    with open(csv_file, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(rows)

if __name__ == "__main__":
    csv_file = './archdaily_projects.csv'
    base_url = 'https://www.archdaily.com/'
    
    with open(csv_file, 'r', encoding='utf-8') as file:

        reader = csv.reader(file)

        header = next(reader)
        status_index = header.index('status')
        
        rows = list(reader)

        for row in rows:
            project_code = row[0]
            status = row[status_index]
            
            # Skip projects with status "downloaded", "error", or "incomplete"
            if status in ['downloaded', 'error', 'incomplete', 'duplicate', 'delete']:
                print(f"Skipping project {project_code} as it is marked as {status}.")
                continue
            
            project_link = base_url + project_code
            print(f"Scraping project: {project_link}")
            status = project_scraper(driver, project_link)
            

            update_csv_status(csv_file, project_code, status)
            remove_csv_status(csv_file)

            
            # # Only download images if status is not "incomplete"
            # if status != "incomplete" and status != "error":
            #     project_image_downloader(driver, project_link)


In [13]:
import time
import requests
import json
import os
import re # For sanitizing filenames
from urllib.parse import urlparse, unquote, urljoin
from pathlib import Path
from bs4 import BeautifulSoup
import logging
from typing import List, Dict, Tuple, Optional # Added type hinting

# --- Selenium Imports ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import traceback # For detailed error printing

# Configure logging (can be shared)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Quieten logs from underlying libraries for cleaner output during normal operation
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("selenium").setLevel(logging.WARNING)
logging.getLogger("webdriver_manager").setLevel(logging.WARNING)


# --- scrape_gallery_thumbnails function ---
def scrape_gallery_thumbnails(url: str) -> Tuple[bool, List[Dict[str, str]]]:
    """
    Scrapes gallery thumbnail information (URL and title) from an Archdaily project page.

    Args:
        url (str): The URL of the Archdaily project page

    Returns:
        tuple: (bool, list) - Success status and list of gallery items [{'href': str, 'title': str}]
    """
    gallery_items: List[Dict[str, str]] = []
    driver = None

    try:
        print("Initializing WebDriver...")
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--log-level=3')
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        os.environ['WDM_LOG_LEVEL'] = '0'

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("WebDriver Initialized.")

        print(f"Fetching project page: {url}")
        driver.get(url)

        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "gallery-thumbs"))
            )
            print("Gallery thumbnails section loaded.")
        except TimeoutException:
            print("Timeout waiting for gallery thumbnails section (gallery-thumbs) to load.")
            pass

        initial_page_source = driver.page_source
        soup = BeautifulSoup(initial_page_source, "html.parser")

        thumbnail_items = soup.find_all("li", class_="gallery-thumbs-item")
        if not thumbnail_items:
             print("Could not find any 'gallery-thumbs-item' list items.")
             return False, gallery_items

        print(f"Found {len(thumbnail_items)} potential thumbnail items to process.")
        success = False

        for i, li in enumerate(thumbnail_items):
            try:
                link = li.find("a", class_="gallery-thumbs-link")
                if not link: continue
                href = link.get("href")
                if not href: continue

                full_href = urljoin(url, href)
                title = link.get("title", "").strip()
                if not title:
                    img = link.find("img")
                    if img: title = img.get("alt", "").strip()
                title = title if title else f"Untitled_{i+1}"

                if not full_href or full_href == url: continue

                item = {"href": full_href, "title": title}
                gallery_items.append(item)
                success = True
            except Exception as e:
                print(f"  Error processing thumbnail item {i+1}: {str(e)}")
                continue

        print(f"\nGallery thumbnail scraping complete. Successfully extracted {len(gallery_items)} items.")
        return success, gallery_items

    except Exception as e:
        print(f"An unexpected error occurred in scrape_gallery_thumbnails: {str(e)}")
        traceback.print_exc()
        return False, gallery_items

    finally:
        if driver:
            print("Quitting WebDriver...")
            try:
                driver.quit()
                print("WebDriver quit successfully.")
            except Exception as e: print(f"Error quitting WebDriver: {e}")


# --- download_gallery_image function ---
def download_gallery_image(page_url: str, save_directory: Path, base_filename: str) -> Tuple[bool, Optional[List[str]], Optional[str], Optional[str]]:
    """
    Downloads the specific image from an ArchDaily gallery page URL, determines its
    extension, saves it to the specified directory with the base filename + extension,
    and returns metadata including the final filename.

    Args:
        page_url: The full URL of the specific ArchDaily gallery image page.
        save_directory: The pathlib.Path object representing the directory to save the image.
        base_filename: The base name for the image file (e.g., "projectID_index").

    Returns:
        A tuple containing:
        - success (bool): True if download and parsing were successful, False otherwise.
        - tags (list[str] | None): A list of tag names for the image, or None on failure.
        - caption (str | None): The caption text for the image, or None on failure.
        - final_filename (str | None): The actual filename used for saving (including extension), or None.
    """
    success = False
    tags = None
    caption = None
    target_image_info = None
    final_filename = None

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        logging.info(f"Fetching HTML from: {page_url}")
        response = requests.get(page_url, headers=headers, timeout=15)
        response.raise_for_status()
        logging.info("HTML fetched successfully.")

        soup = BeautifulSoup(response.text, 'html.parser')
        gallery_div = soup.find('div', id='gallery-items')
        if not gallery_div:
            logging.error("Could not find gallery container ('div#gallery-items').")
            return success, tags, caption, final_filename

        data_images_json = gallery_div.get('data-images')
        default_image_id_fragment = gallery_div.get('data-id')
        if not data_images_json:
            logging.error("Could not find 'data-images' attribute.")
            return success, tags, caption, final_filename

        try:
            image_data = json.loads(data_images_json)
            if not isinstance(image_data, list) or not image_data:
                 logging.error("'data-images' did not contain a valid list.")
                 return success, tags, caption, final_filename
        except json.JSONDecodeError as e:
            logging.error(f"Failed to parse JSON: {e}")
            return success, tags, caption, final_filename

        parsed_url = urlparse(page_url)
        url_fragment = parsed_url.fragment
        target_image_info = None # Reset before searching

        if url_fragment:
            target_id = url_fragment.split('-')[0]
            for img_info in image_data:
                if 'link' in img_info and target_id in img_info['link']:
                     target_image_info = img_info
                     break
            if not target_image_info:
                 logging.warning(f"Fragment ID '{target_id}' not matched. Falling back.")

        if not target_image_info: # Fallback to default ID
            if default_image_id_fragment:
                for img_info in image_data:
                     if 'link' in img_info and default_image_id_fragment in img_info['link']:
                         target_image_info = img_info
                         break
                if not target_image_info:
                     logging.warning(f"Default ID {default_image_id_fragment} not matched. Using first image.")
                     if image_data: target_image_info = image_data[0]
            else:
                logging.warning("No fragment/default ID. Using first image.")
                if image_data: target_image_info = image_data[0]

        if not target_image_info:
             logging.error("Could not determine target image.")
             return success, tags, caption, final_filename

        image_url = target_image_info.get('url_large') or target_image_info.get('url_slideshow')
        caption = target_image_info.get('caption', 'No Caption')
        raw_tags = target_image_info.get('tags', [])
        tags = [tag.get('name', '').strip() for tag in raw_tags if isinstance(tag, dict) and 'name' in tag]

        if not image_url:
            logging.error(f"Target image missing URL (Link: {target_image_info.get('link')}).")
            return success, tags, caption, final_filename

        parsed_image_url = urlparse(image_url)
        image_path_part = Path(unquote(parsed_image_url.path))
        extension = image_path_part.suffix
        if not extension or len(extension) > 5:
             extension = ".jpg"
             logging.warning(f"Using default extension .jpg for {image_url}")

        final_filename = f"{base_filename}{extension}"
        save_path = save_directory / final_filename
        logging.info(f"Image will be saved as: {save_path}")

        logging.info(f"Downloading image from {image_url}...")
        img_response = requests.get(image_url, stream=True, headers=headers, timeout=30)
        img_response.raise_for_status()

        with open(save_path, 'wb') as f:
            for chunk in img_response.iter_content(chunk_size=8192):
                f.write(chunk)

        logging.info(f"Image successfully downloaded.")
        success = True

    except requests.exceptions.Timeout as e: logging.error(f"Request timed out: {e}")
    except requests.exceptions.HTTPError as e: logging.error(f"HTTP Error: {e.response.status_code} - {e}")
    except requests.exceptions.RequestException as e: logging.error(f"Network error: {e}")
    except FileNotFoundError: logging.error(f"Error accessing path {save_path}.")
    except Exception as e: logging.error(f"Unexpected error: {e}", exc_info=False)

    finally:
        return success, tags, caption, final_filename


# --- Main Processing Function ---
def project_image_downloader(project_url: str) -> Optional[Path]:
    """
    Processes a single ArchDaily project: scrapes thumbnails, downloads images,
    and saves metadata to a JSON file within a project-specific folder.

    Args:
        project_url (str): The URL of the ArchDaily project page.

    Returns:
        Optional[Path]: The Path object to the created project folder if successful,
                       otherwise None.
    """
    print(f"\n=== Processing Project: {project_url} ===")

    # --- Extract Project ID and Set Up Base Folder ---
    project_id = None
    try:
        path_parts = [part for part in urlparse(project_url).path.strip('/').split('/') if part]
        for part in path_parts:
            if part.isdigit():
                project_id = part
                break
    except Exception as e:
        print(f"[ERROR] Error extracting project ID from URL '{project_url}': {e}")
        return None # Critical failure

    if not project_id:
        print(f"[ERROR] Could not extract numeric project ID from URL: {project_url}.")
        return None # Critical failure

    base_download_folder = Path(project_id)
    print(f"Project ID detected: {project_id}")
    print(f"Base download directory set to: '{base_download_folder.resolve()}'")

    # Create the base directory
    try:
        base_download_folder.mkdir(parents=True, exist_ok=True)
        print(f"Ensured base directory exists.")
    except OSError as e:
        print(f"[ERROR] Error creating base directory '{base_download_folder}': {e}. Check permissions.")
        return None # Critical failure

    # --- Start Scraping ---
    print(f"\n--- Starting scraping process for thumbnails ---")
    success_scrape, gallery_items = scrape_gallery_thumbnails(project_url)

    if not success_scrape or not gallery_items:
        print("\n--- Thumbnail scraping failed or no items found. Process halted. ---")
        return None # Indicate failure if scraping doesn't yield items
    else:
        print(f"\n--- Starting download process for {len(gallery_items)} items ---")

        all_metadata = [] # Initialize list for JSON

        # --- Iterate and Download ---
        successful_downloads = 0
        for index, item in enumerate(gallery_items):
            item_url = item.get('href')
            item_title = item.get('title', f'image_{index+1}')

            print(f"\n>>> Processing Item {index+1}/{len(gallery_items)}: '{item_title}' <<<")
            # print(f"    Source URL: {item_url}") # Reduced verbosity

            if not item_url:
                print("    Skipping item: Missing URL.")
                continue

            base_filename = f"{project_id}_{index+1:02d}"

            # Call download function
            success_download, tags, caption, final_filename = download_gallery_image(
                item_url,
                base_download_folder, # Pass directory Path object
                base_filename         # Pass base name
            )

            # --- Report and Collect Metadata ---
            if success_download and final_filename:
                print(f"    Download Status: SUCCESS")
                print(f"      Saved as: {final_filename}")
                # print(f"      Caption: {caption}") # Reduced verbosity
                # print(f"      Tags: {tags}")       # Reduced verbosity
                metadata_entry = {
                    "filename": final_filename,
                    "tags": tags if tags else [],
                    "caption": caption if caption else ""
                }
                all_metadata.append(metadata_entry)
                successful_downloads += 1
            else:
                print(f"    Download Status: FAILED")
                # Optionally log partial metadata even on failure
                # if caption or tags: print(f"      Caption/Tags (if parsed): {caption} / {tags}")

            # Add delay
            # print("    Waiting 1 second before next item...") # Reduced verbosity
            time.sleep(0.5) # Slightly shorter delay

        print(f"\n--- Download process complete. {successful_downloads}/{len(gallery_items)} images downloaded successfully. ---")

        # --- Save Metadata JSON ---
        if all_metadata:
            metadata_filepath = base_download_folder / f"{project_id}.json"
            print(f"\n--- Saving metadata for {len(all_metadata)} items to {metadata_filepath} ---")
            try:
                with open(metadata_filepath, 'w', encoding='utf-8') as f:
                    json.dump(all_metadata, f, indent=4, ensure_ascii=False)
                print("Metadata successfully saved.")
            except IOError as e:
                print(f"[ERROR] Error saving metadata JSON file: {e}")
        else:
            print("\n--- No metadata collected (or all downloads failed), skipping JSON file creation. ---")

        print(f"\n=== Finished processing project {project_id} ===")
        return base_download_folder # Return the path on success

# --- Main Execution (Example Call) ---
if __name__ == "__main__":
    target_project_url = "https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects"
    # You can add more URLs to a list and loop through them:
    # target_urls = [
    #     "https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects",
    #     "https://www.archdaily.com/977011/pinghe-bibliotheater-open-architecture"
    # ]
    # for url in target_urls:
    #    result_folder = process_archdaily_project(url)
    #    if result_folder:
    #        print(f"Successfully processed '{url}', files saved in '{result_folder.resolve()}'")
    #    else:
    #        print(f"Failed to process '{url}'")
    #    print("-" * 50) # Separator

    # Single URL example:
    result_folder = project_image_downloader(target_project_url)

    if result_folder:
        print(f"\n\nOverall process completed successfully.")
        print(f"Project files saved in: '{result_folder.resolve()}'")
    else:
        print(f"\n\nOverall process failed for {target_project_url}.")

2025-04-06 23:46:58,174 - INFO - ====== WebDriver manager ======



=== Processing Project: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects ===
Project ID detected: 904658
Base download directory set to: 'D:\999_RAG\crawler\904658'
Ensured base directory exists.

--- Starting scraping process for thumbnails ---
Initializing WebDriver...


2025-04-06 23:46:58,946 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-06 23:46:59,335 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-06 23:46:59,703 - INFO - Driver [C:\Users\15717\.wdm\drivers\chromedriver\win64\134.0.6998.165\chromedriver-win32/chromedriver.exe] found in cache


WebDriver Initialized.
Fetching project page: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects
Gallery thumbnails section loaded.
Found 24 potential thumbnail items to process.

Gallery thumbnail scraping complete. Successfully extracted 24 items.
Quitting WebDriver...
WebDriver quit successfully.


2025-04-06 23:47:30,047 - INFO - Fetching HTML from: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo


...

    Download Status: SUCCESS
      Saved as: 904658_23.jpg


2025-04-06 23:49:15,437 - INFO - Fetching HTML from: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1ab2cf197cc96d0000331-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-water-diagram



>>> Processing Item 24/24: 'Phillip and Patricia Frost Museum of Science / Grimshaw Architects' <<<


2025-04-06 23:49:17,430 - INFO - HTML fetched successfully.
2025-04-06 23:49:17,440 - INFO - Image will be saved as: 904658\904658_24.jpg
2025-04-06 23:49:17,440 - INFO - Downloading image from https://images.adsttc.com/media/images/5bd1/ab2c/f197/cc96/d000/0331/large_jpg/03_CD_2008-1221_water.jpg?1540467493...
2025-04-06 23:49:19,941 - INFO - Image successfully downloaded.


    Download Status: SUCCESS
      Saved as: 904658_24.jpg

--- Download process complete. 24/24 images downloaded successfully. ---

--- Saving metadata for 24 items to 904658\904658.json ---
Metadata successfully saved.

=== Finished processing project 904658 ===


Overall process completed successfully.
Project files saved in: 'D:\999_RAG\crawler\904658'


In [14]:
import csv
import os
import requests
import time
import re
import json
from urllib.parse import urlparse, unquote, urljoin
from pathlib import Path
from bs4 import BeautifulSoup
import logging
from typing import List, Dict, Tuple, Optional

# --- Selenium Imports ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver # Import WebDriver type hint
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import traceback # For detailed error printing

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("selenium").setLevel(logging.WARNING)
logging.getLogger("webdriver_manager").setLevel(logging.WARNING)

# --- Helper Functions for project_scraper ---
def extract_text_with_spacing(element):
    """Recursively extracts text preserving approximate spacing."""
    result = ""
    if element is None:
        return result
    for content in element.contents:
        if hasattr(content, 'name') and content.name: # if it's a tag
            result += extract_text_with_spacing(content)
        elif hasattr(content, 'string'): # if it's NavigableString (text)
            result += content.string
        # Handle cases where content might be None or other types
    return result

def purge_description(description_list):
    """Cleans the list of description paragraphs."""
    # Remove items with 3 or fewer words
    description_list = [x for x in description_list if len(x.split()) > 3]
    # Remove specific boilerplate text
    description_list = [x for x in description_list if "You'll now receive updates based on what you follow!" not in x]
    # Remove "Check the" items
    description_list = [x for x in description_list if not x.startswith("Check the")]
    # Remove "Save this picture!" items
    description_list = [x for x in description_list if "Save this picture!" not in x]
    # Remove duplicates while preserving order
    seen = set()
    description_list = [x for x in description_list if not (x in seen or seen.add(x))]
    return description_list

# --- project_scraper function (MODIFIED: No driver needed, saves details JSON) ---
def project_scraper(project_link: str) -> str:
    """
    Scrapes project details (metadata like architects, area, year, description)
    and saves them to [project_id]_details.json.
    Uses Requests, not Selenium.

    Args:
        project_link (str): The URL of the ArchDaily project page.

    Returns:
        str: Status ("downloaded" on success, "error" on failure).
    """
    print(f"--- Scraping Project Details for: {project_link} ---")
    headers = { # Add headers for requests
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9'
    }
    try:
        response = requests.get(project_link, headers=headers, timeout=20)
        response.raise_for_status() # Check for HTTP errors
    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Network error fetching details for {project_link}: {e}")
        return "error"

    try:
        soup = BeautifulSoup(response.content, 'html.parser')
        print('Project details HTML parsed.')

        # Extract project ID from link for saving
        project_id = None
        try:
            path_parts = [part for part in urlparse(project_link).path.strip('/').split('/') if part]
            for part in path_parts:
                if part.isdigit():
                    project_id = part
                    break
        except Exception:
            pass # Ignore if ID extraction fails here, use fallback

        if not project_id:
             # Fallback: Try getting ID differently or use a placeholder
             project_id_match = re.search(r'/(\d+)/', project_link)
             if project_id_match:
                 project_id = project_id_match.group(1)
             else:
                 print("[WARN] Could not determine project ID for filename.")
                 project_id = "unknown_project" # Placeholder ID

        # --- Data Extraction Logic (similar to original) ---
        data = {'categories': [], 'city': None, 'country': None, 'architects': [], 'area': None, 'year': None, 'description': None}
        title_tag = soup.find(class_='afd-title-big--bmargin-big')
        project_title = title_tag.get_text(strip=True).split('/')[0].strip() if title_tag else "Untitled Project"

        category_container = soup.find(class_='afd-specs__header-category')
        if category_container: data['categories'] = [a.text.strip() for a in category_container.find_all('a')]

        location_container = soup.find(class_='afd-specs__header-location')
        if location_container:
            location_parts = location_container.get_text(strip=True).split(',')
            if location_parts: data['city'] = location_parts[0].strip()
            country_link = location_container.find('a')
            if country_link: data['country'] = country_link.text.strip()

        for item in soup.select('.afd-specs__item'):
            key_tag = item.find(class_='afd-specs__key')
            value_tag = item.find(class_='afd-specs__value')
            if not key_tag or not value_tag: continue
            key_text = key_tag.get_text(strip=True)
            if 'Architects' in key_text: data['architects'] = [a.text.strip() for a in value_tag.find_all('a')]
            elif 'Area' in key_text: data['area'] = value_tag.text.replace('m²', '').strip()
            elif 'Year' in key_text: data['year'] = value_tag.text.strip()
            # Add more fields here if needed (e.g., Manufacturers, etc.)

        # Description extraction
        description_paragraphs = []
        desc_section = soup.find('div', class_='the-content') # Target the main content div
        if desc_section:
            for paragraph in desc_section.find_all('p', recursive=False): # Find direct children paragraphs
                extracted_text = extract_text_with_spacing(paragraph)
                normalized_text = re.sub(r'\s+', ' ', extracted_text).strip()
                if normalized_text: # Avoid empty strings
                     description_paragraphs.append(normalized_text)
        else: # Fallback if specific content div isn't found
             for paragraph in soup.find_all('p'):
                 extracted_text = extract_text_with_spacing(paragraph)
                 normalized_text = re.sub(r'\s+', ' ', extracted_text).strip()
                 if normalized_text:
                     description_paragraphs.append(normalized_text)

        data['description'] = purge_description(description_paragraphs) # Clean the description list
        # --- End Data Extraction ---

        # Construct final dict
        project_dict = {
            'Project ID': project_id,
            'Project Title': project_title,
            'Categories': data['categories'] or None,
            'City': data['city'],
            'Country': data['country'],
            'Architects': data['architects'] or None,
            'Area': f"{data['area']} m²" if data['area'] else None,
            'Year': data['year'],
            'Project URL': project_link,
            'Description': data['description'] or None
        }
        project_dict = {k: v for k, v in project_dict.items() if v} # Remove keys with None/empty values

        # --- Save details JSON ---
        details_filename = f'{project_id}_details.json'
        # Ensure the base directory exists (needed if images aren't downloaded first)
        Path(project_id).mkdir(parents=True, exist_ok=True)
        details_filepath = Path(project_id) / details_filename

        with open(details_filepath, 'w', encoding='utf-8') as f:
            json.dump(project_dict, f, ensure_ascii=False, indent=2)

        print(f'Project details saved to: {details_filepath}')
        return "downloaded" # Use status consistent with original script

    except Exception as e:
        print(f"[ERROR] Error processing details for {project_link}: {e}")
        traceback.print_exc()
        return "error"


# --- scrape_gallery_thumbnails function (MODIFIED: Accepts driver) ---
def scrape_gallery_thumbnails(driver: WebDriver, url: str) -> Tuple[bool, List[Dict[str, str]]]:
    """
    Scrapes gallery thumbnail information (URL and title) using a provided WebDriver.
    """
    gallery_items: List[Dict[str, str]] = []
    # WebDriver is now passed in, no internal creation/destruction

    try:
        print(f"Fetching project page with Selenium: {url}")
        driver.get(url)

        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "gallery-thumbs"))
            )
            print("Gallery thumbnails section loaded.")
        except TimeoutException:
            print("Timeout waiting for gallery thumbnails section.")
            # Consider if this should be a hard fail
            pass # Attempt to parse anyway

        # Use the passed driver to get page source
        initial_page_source = driver.page_source
        soup = BeautifulSoup(initial_page_source, "html.parser")

        thumbnail_items = soup.find_all("li", class_="gallery-thumbs-item")
        if not thumbnail_items:
             print("Could not find any 'gallery-thumbs-item' list items.")
             # It might be valid for a project to have no images/thumbnails
             return True, gallery_items # Return True but empty list

        print(f"Found {len(thumbnail_items)} potential thumbnail items.")
        processed_count = 0
        for i, li in enumerate(thumbnail_items):
            try:
                link = li.find("a", class_="gallery-thumbs-link")
                if not link: continue
                href = link.get("href")
                if not href: continue

                full_href = urljoin(url, href)
                title = link.get("title", "").strip()
                if not title:
                    img = link.find("img")
                    if img: title = img.get("alt", "").strip()
                title = title if title else f"Untitled_{i+1}"

                if not full_href or full_href == url: continue # Skip self-links

                item = {"href": full_href, "title": title}
                gallery_items.append(item)
                processed_count += 1
            except Exception as e:
                print(f"  Error processing thumbnail item {i+1}: {str(e)}")
                continue # Skip this item

        print(f"Gallery thumbnail scraping complete. Successfully extracted {processed_count} items.")
        # Success is true if the process ran, even if 0 items were found/processed
        return True, gallery_items

    except WebDriverException as e:
        print(f"[ERROR] WebDriver error during thumbnail scraping for {url}: {e}")
        return False, gallery_items
    except Exception as e:
        print(f"[ERROR] Unexpected error in scrape_gallery_thumbnails for {url}: {e}")
        traceback.print_exc()
        return False, gallery_items
    # No driver.quit() here - managed globally


# --- download_gallery_image function (Unchanged) ---
def download_gallery_image(page_url: str, save_directory: Path, base_filename: str) -> Tuple[bool, Optional[List[str]], Optional[str], Optional[str]]:
    # --- (Content of this function remains the same as in the previous version) ---
    # ... (Keep the function body exactly as provided in the previous step) ...
    # ... (It uses requests, parses JSON, downloads image, returns tuple) ...
    success = False
    tags = None
    caption = None
    target_image_info = None
    final_filename = None

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        logging.info(f"Fetching image page HTML: {page_url}")
        response = requests.get(page_url, headers=headers, timeout=15)
        response.raise_for_status()
        logging.info("HTML fetched.")

        soup = BeautifulSoup(response.text, 'html.parser')
        gallery_div = soup.find('div', id='gallery-items')
        if not gallery_div:
            logging.error("No gallery container ('div#gallery-items').")
            return success, tags, caption, final_filename

        data_images_json = gallery_div.get('data-images')
        default_image_id_fragment = gallery_div.get('data-id')
        if not data_images_json:
            logging.error("No 'data-images' attribute.")
            return success, tags, caption, final_filename

        try:
            image_data = json.loads(data_images_json)
            if not isinstance(image_data, list) or not image_data:
                 logging.error("'data-images' not a valid list.")
                 return success, tags, caption, final_filename
        except json.JSONDecodeError as e:
            logging.error(f"JSON parse error: {e}")
            return success, tags, caption, final_filename

        parsed_url = urlparse(page_url)
        url_fragment = parsed_url.fragment
        target_image_info = None

        if url_fragment:
            target_id = url_fragment.split('-')[0]
            for img_info in image_data:
                if 'link' in img_info and target_id in img_info['link']:
                     target_image_info = img_info
                     break
            if not target_image_info:
                 logging.warning(f"Fragment '{target_id}' not matched.")

        if not target_image_info: # Fallback
            if default_image_id_fragment:
                for img_info in image_data:
                     if 'link' in img_info and default_image_id_fragment in img_info['link']:
                         target_image_info = img_info
                         break
                if not target_image_info:
                     logging.warning(f"Default ID {default_image_id_fragment} not matched.")
                     if image_data: target_image_info = image_data[0] # Use first
            else:
                logging.warning("No fragment/default ID. Using first image.")
                if image_data: target_image_info = image_data[0] # Use first

        if not target_image_info:
             logging.error("Cannot determine target image.")
             return success, tags, caption, final_filename

        image_url = target_image_info.get('url_large') or target_image_info.get('url_slideshow')
        caption = target_image_info.get('caption', 'No Caption')
        raw_tags = target_image_info.get('tags', [])
        tags = [tag.get('name', '').strip() for tag in raw_tags if isinstance(tag, dict) and 'name' in tag]

        if not image_url:
            logging.error(f"Target image missing URL (Link: {target_image_info.get('link')}).")
            return success, tags, caption, final_filename

        parsed_image_url = urlparse(image_url)
        image_path_part = Path(unquote(parsed_image_url.path))
        extension = image_path_part.suffix
        if not extension or len(extension) > 5:
             extension = ".jpg"
             logging.warning(f"Using default .jpg for {image_url}")

        final_filename = f"{base_filename}{extension}"
        save_path = save_directory / final_filename
        logging.info(f"Target save path: {save_path}")

        logging.info(f"Downloading image: {image_url}...")
        img_response = requests.get(image_url, stream=True, headers=headers, timeout=30)
        img_response.raise_for_status()

        with open(save_path, 'wb') as f:
            for chunk in img_response.iter_content(chunk_size=8192):
                f.write(chunk)

        logging.info(f"Image download SUCCESS.")
        success = True

    except requests.exceptions.Timeout: logging.error("Request timed out.")
    except requests.exceptions.HTTPError as e: logging.error(f"HTTP Error: {e.response.status_code}")
    except requests.exceptions.RequestException as e: logging.error(f"Network error: {e}")
    except IOError as e: logging.error(f"File IO error: {e}")
    except Exception as e: logging.error(f"Unexpected error: {e}", exc_info=False)

    finally:
        return success, tags, caption, final_filename


# --- Main Image Processing Function (MODIFIED: Accepts driver, saves images JSON) ---
def process_project_images(driver: WebDriver, project_url: str) -> Tuple[bool, Optional[Path]]:
    """
    Processes images for a single ArchDaily project using a provided WebDriver:
    scrapes thumbnails, downloads images, saves image metadata JSON.

    Args:
        driver (WebDriver): The shared Selenium WebDriver instance.
        project_url (str): The URL of the ArchDaily project page.

    Returns:
        Tuple[bool, Optional[Path]]: Overall success status for image processing,
                                     and the Path object to the project folder if successful.
    """
    print(f"--- Processing Project Images for: {project_url} ---")
    overall_image_success = False # Track if at least one image downloads

    # --- Extract Project ID and Set Up Base Folder ---
    project_id = None
    try:
        path_parts = [part for part in urlparse(project_url).path.strip('/').split('/') if part]
        for part in path_parts:
            if part.isdigit():
                project_id = part
                break
    except Exception as e:
        print(f"[ERROR] Error extracting project ID for images: {e}")
        return overall_image_success, None

    if not project_id:
        print(f"[ERROR] Could not extract project ID for images: {project_url}.")
        return overall_image_success, None

    base_download_folder = Path(project_id)
    print(f"Image Project ID: {project_id}")
    print(f"Image download directory: '{base_download_folder.resolve()}'")

    try:
        base_download_folder.mkdir(parents=True, exist_ok=True)
    except OSError as e:
        print(f"[ERROR] Error creating/accessing image directory '{base_download_folder}': {e}.")
        return overall_image_success, None

    # --- Scrape Thumbnails using shared driver ---
    print(f"Scraping thumbnails...")
    # Pass the shared driver to the modified scrape_gallery_thumbnails
    success_scrape, gallery_items = scrape_gallery_thumbnails(driver, project_url)

    if not success_scrape:
        print("Thumbnail scraping failed. Cannot proceed with image downloads.")
        return overall_image_success, base_download_folder # Return folder path but False status
    if not gallery_items:
        print("No gallery items found to download.")
        # Consider this a success in terms of processing, just nothing to download
        return True, base_download_folder

    # --- Iterate and Download ---
    print(f"--- Starting image downloads for {len(gallery_items)} items ---")
    all_metadata = []
    successful_downloads = 0
    for index, item in enumerate(gallery_items):
        item_url = item.get('href')
        item_title = item.get('title', f'image_{index+1}')

        print(f"\n>>> Downloading Item {index+1}/{len(gallery_items)}: '{item_title}' <<<")

        if not item_url:
            print("    Skipping: Missing URL.")
            continue

        base_filename = f"{project_id}_{index+1:02d}"

        success_download, tags, caption, final_filename = download_gallery_image(
            item_url,
            base_download_folder,
            base_filename
        )

        if success_download and final_filename:
            print(f"    SUCCESS: Saved as {final_filename}")
            metadata_entry = {
                "filename": final_filename,
                "tags": tags if tags else [],
                "caption": caption if caption else ""
            }
            all_metadata.append(metadata_entry)
            successful_downloads += 1
            overall_image_success = True # Mark overall success if at least one downloads
        else:
            print(f"    FAILED")

        time.sleep(0.5) # Delay

    print(f"\n--- Image download process complete: {successful_downloads}/{len(gallery_items)} successful. ---")

    # --- Save Image Metadata JSON ---
    if all_metadata:
        metadata_filepath = base_download_folder / f"{project_id}_images.json" # New filename
        print(f"Saving image metadata to {metadata_filepath}")
        try:
            with open(metadata_filepath, 'w', encoding='utf-8') as f:
                json.dump(all_metadata, f, indent=4, ensure_ascii=False)
            print("Image metadata saved.")
        except IOError as e:
            print(f"[ERROR] Failed to save image metadata JSON: {e}")
    else:
        print("No image metadata collected.")

    print(f"--- Finished processing images for project {project_id} ---")
    # Return overall success status and the folder path
    return overall_image_success, base_download_folder


# --- CSV Handling Functions (Fixed typo) ---
def update_csv_status(csv_file, project_code, status):
    rows = []
    try:
        with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            header = next(reader)
            try:
                 status_index = header.index('status')
            except ValueError:
                 print(f"[ERROR] 'status' column not found in CSV header: {header}")
                 return # Cannot update status
            rows.append(header)
            for row in reader:
                if row and row[0] == project_code: # Check if row is not empty
                    # Ensure row has enough columns before accessing status_index
                    if len(row) > status_index:
                        row[status_index] = status
                    else:
                        # Handle rows that might be shorter than the header
                        print(f"[WARN] Row for {project_code} is too short, cannot update status.")
                        # Optionally append empty strings to match header length
                        row.extend([''] * (len(header) - len(row)))
                        row[status_index] = status # Try updating now
                rows.append(row)
    except FileNotFoundError:
        print(f"[ERROR] CSV file not found: {csv_file}")
        return
    except Exception as e:
        print(f"[ERROR] Error reading CSV file {csv_file}: {e}")
        return

    try:
        # Use temporary file for safer writing
        temp_file = Path(csv_file).with_suffix('.tmp')
        with open(temp_file, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(rows)
        # Replace original file with temporary file
        os.replace(temp_file, csv_file)
        print(f"Updated status for {project_code} to '{status}' in {csv_file}")
    except Exception as e:
        print(f"[ERROR] Error writing updated CSV file {csv_file}: {e}")
        if temp_file.exists():
             os.remove(temp_file) # Clean up temp file on error


def remove_csv_status(csv_file):
    # remove all the status in the csv file
    rows = []
    try:
        with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            header = next(reader)
            try:
                 status_index = header.index('status')
            except ValueError:
                 print(f"[ERROR] 'status' column not found in CSV header: {header}")
                 return
            rows.append(header)
            for row in reader:
                 if row: # Check if row is not empty
                      if len(row) > status_index:
                           row[status_index] = "" # Clear the status
                      else:
                           # Handle short rows if necessary, or just skip status update
                           pass
                 rows.append(row) # Append row even if it was short/empty
    except FileNotFoundError:
        print(f"[ERROR] CSV file not found: {csv_file}")
        return
    except Exception as e:
        print(f"[ERROR] Error reading CSV file {csv_file} for status removal: {e}")
        return

    try:
        # Use temporary file for safer writing
        temp_file = Path(csv_file).with_suffix('.tmp')
        with open(temp_file, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(rows)
        os.replace(temp_file, csv_file)
        print(f"Cleared all statuses in {csv_file}")
    except Exception as e:
        print(f"[ERROR] Error writing CSV file {csv_file} after clearing statuses: {e}")
        if temp_file.exists():
             os.remove(temp_file)


# --- Main Execution Logic (Integrated) ---
if __name__ == "__main__":
    csv_file = './archdaily_projects.csv'
    base_url = 'https://www.archdaily.com' # Define base_url needed for link construction

    # --- Initialize Shared WebDriver ---
    driver = None
    try:
        print("Initializing Shared WebDriver for the session...")
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--log-level=3')
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        os.environ['WDM_LOG_LEVEL'] = '0'

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("Shared WebDriver Initialized.")

        # --- Optional: Reset all statuses before starting ---
        # print(f"\nClearing previous statuses in {csv_file}...")
        # remove_csv_status(csv_file)
        # print("Statuses cleared.")
        # --- End Optional Reset ---


        # --- Read CSV and Process Projects ---
        print(f"\nReading projects from {csv_file}...")
        try:
            with open(csv_file, 'r', encoding='utf-8') as file:
                reader = csv.reader(file)
                header = next(reader)
                try:
                    status_index = header.index('status')
                    # Assuming project code is the first column
                    code_index = 0
                except ValueError:
                    print("[ERROR] CSV file must contain 'status' column. Exiting.")
                    exit()

                rows = list(reader) # Read all rows into memory for easier processing

        except FileNotFoundError:
            print(f"[ERROR] CSV file not found: {csv_file}. Exiting.")
            exit()
        except Exception as e:
            print(f"[ERROR] Failed to read CSV file {csv_file}: {e}. Exiting.")
            exit()

        print(f"Found {len(rows)} projects in CSV.")

        for i, row in enumerate(rows):
            if not row: # Skip empty rows
                 print(f"Skipping empty row {i+1}")
                 continue
            try:
                project_code = row[code_index]
                status = row[status_index] if len(row) > status_index else ""
            except IndexError:
                 print(f"[WARN] Skipping row {i+1} due to insufficient columns: {row}")
                 continue

            print(f"\n{'='*10} Processing Project {i+1}/{len(rows)}: {project_code} {'='*10}")

            # Skip based on status
            skip_statuses = ['downloaded', 'error', 'incomplete', 'duplicate', 'delete'] # Add more if needed
            if status.lower() in skip_statuses:
                print(f"Skipping project {project_code} (Status: '{status}').")
                continue

            # Construct full project link
            # Ensure project_code doesn't have leading/trailing slashes if base_url ends with one
            project_link = f"{base_url.strip('/')}/{project_code.strip('/')}"
            print(f"Project URL: {project_link}")

            # --- Step 1: Scrape Project Details ---
            scraper_status = project_scraper(project_link) # Uses Requests

            # --- Step 2: Process Project Images (if details scraped successfully) ---
            downloader_overall_success = False
            if scraper_status == "downloaded":
                # Pass the shared driver here
                downloader_overall_success, _ = process_project_images(driver, project_link)
            else:
                print("Skipping image download because project details scraping failed.")

            # --- Step 3: Determine Final Status and Update CSV ---
            final_status = "error" # Default to error
            if scraper_status == "downloaded":
                if downloader_overall_success:
                    final_status = "downloaded" # Both parts succeeded
                else:
                    # Scraper OK, but downloader failed or found no images to download successfully
                    final_status = "incomplete"
            # Keep "error" if scraper_status was "error"

            print(f"Final status for {project_code}: {final_status}")
            update_csv_status(csv_file, project_code, final_status)

            # Optional short delay between projects
            time.sleep(1)

        print("\nFinished processing all projects from CSV.")

    except Exception as e:
        print(f"\nAn unexpected error occurred during the main processing loop: {e}")
        traceback.print_exc()
    finally:
        # --- Clean Up Shared WebDriver ---
        if driver:
            print("\nQuitting Shared WebDriver...")
            try:
                driver.quit()
                print("Shared WebDriver quit successfully.")
            except Exception as e:
                print(f"Error quitting Shared WebDriver: {e}")

2025-04-06 23:55:54,607 - INFO - ====== WebDriver manager ======


Initializing Shared WebDriver for the session...


2025-04-06 23:55:55,369 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-06 23:55:55,730 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-06 23:55:56,066 - INFO - Driver [C:\Users\15717\.wdm\drivers\chromedriver\win64\134.0.6998.165\chromedriver-win32/chromedriver.exe] found in cache


Shared WebDriver Initialized.

Reading projects from ./archdaily_projects.csv...
Found 8 projects in CSV.

========== Processing Project 1/8: 904658 ==========
Project URL: https://www.archdaily.com/904658
--- Scraping Project Details for: https://www.archdaily.com/904658 ---
Project details HTML parsed.
Project details saved to: 904658\904658_details.json
--- Processing Project Images for: https://www.archdaily.com/904658 ---
Image Project ID: 904658
Image download directory: 'D:\999_RAG\crawler\904658'
Scraping thumbnails...
Fetching project page with Selenium: https://www.archdaily.com/904658


2025-04-06 23:56:42,637 - INFO - Fetching image page HTML: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo


...

    SUCCESS: Saved as 801031_22.jpg


2025-04-07 00:16:52,973 - INFO - Fetching image page HTML: https://www.archdaily.com/801031/mathematics-the-winton-gallery-zaha-hadid-architects/58484798e58ece80e80001e7-mathematics-the-winton-gallery-zaha-hadid-architects-design-concept



>>> Downloading Item 23/23: 'Mathematics: The Winton Gallery / Zaha Hadid Architects' <<<


2025-04-07 00:16:55,294 - INFO - HTML fetched.
2025-04-07 00:16:55,303 - INFO - Target save path: 801031\801031_23.jpg
2025-04-07 00:16:55,304 - INFO - Downloading image: https://images.adsttc.com/media/images/5848/4798/e58e/ce80/e800/01e7/large_jpg/Design_concept_01.jpg?1481131909...
2025-04-07 00:16:57,765 - INFO - Image download SUCCESS.


    SUCCESS: Saved as 801031_23.jpg

--- Image download process complete: 23/23 successful. ---
Saving image metadata to 801031\801031_images.json
Image metadata saved.
--- Finished processing images for project 801031 ---
Final status for 801031: downloaded
Updated status for 801031 to 'downloaded' in ./archdaily_projects.csv

Finished processing all projects from CSV.

Quitting Shared WebDriver...
Shared WebDriver quit successfully.


In [ ]:
import csv
import os
import requests
import time
import re
import json
from urllib.parse import urlparse, unquote, urljoin
from pathlib import Path
from bs4 import BeautifulSoup
import logging
from typing import List, Dict, Tuple, Optional

# --- Selenium Imports ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver # Import WebDriver type hint
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import traceback # For detailed error printing

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("selenium").setLevel(logging.WARNING)
logging.getLogger("webdriver_manager").setLevel(logging.WARNING)

# --- Helper Functions for project_scraper ---
def extract_text_with_spacing(element):
    """Recursively extracts text preserving approximate spacing."""
    result = ""
    if element is None:
        return result
    for content in element.contents:
        if hasattr(content, 'name') and content.name: # if it's a tag
            result += extract_text_with_spacing(content)
        elif hasattr(content, 'string'): # if it's NavigableString (text)
            result += content.string
        # Handle cases where content might be None or other types
    return result

def purge_description(description_list):
    """Cleans the list of description paragraphs."""
    # Remove items with 3 or fewer words
    description_list = [x for x in description_list if len(x.split()) > 3]
    # Remove specific boilerplate text
    description_list = [x for x in description_list if "You'll now receive updates based on what you follow!" not in x]
    # Remove "Check the" items
    description_list = [x for x in description_list if not x.startswith("Check the")]
    # Remove "Save this picture!" items
    description_list = [x for x in description_list if "Save this picture!" not in x]
    # Remove duplicates while preserving order
    seen = set()
    description_list = [x for x in description_list if not (x in seen or seen.add(x))]
    return description_list

# --- project_scraper function (MODIFIED: No driver needed, saves details JSON) ---
def project_scraper(project_link: str) -> str:
    """
    Scrapes project details (metadata like architects, area, year, description)
    and saves them to [project_id]_details.json.
    Uses Requests, not Selenium.

    Args:
        project_link (str): The URL of the ArchDaily project page.

    Returns:
        str: Status ("downloaded" on success, "error" on failure).
    """
    print(f"--- Scraping Project Details for: {project_link} ---")
    headers = { # Add headers for requests
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9'
    }
    try:
        response = requests.get(project_link, headers=headers, timeout=20)
        response.raise_for_status() # Check for HTTP errors
    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Network error fetching details for {project_link}: {e}")
        return "error"

    try:
        soup = BeautifulSoup(response.content, 'html.parser')
        print('Project details HTML parsed.')

        # Extract project ID from link for saving
        project_id = None
        try:
            path_parts = [part for part in urlparse(project_link).path.strip('/').split('/') if part]
            for part in path_parts:
                if part.isdigit():
                    project_id = part
                    break
        except Exception:
            pass # Ignore if ID extraction fails here, use fallback

        if not project_id:
             # Fallback: Try getting ID differently or use a placeholder
             project_id_match = re.search(r'/(\d+)/', project_link)
             if project_id_match:
                 project_id = project_id_match.group(1)
             else:
                 print("[WARN] Could not determine project ID for filename.")
                 project_id = "unknown_project" # Placeholder ID

        # --- Data Extraction Logic (similar to original) ---
        data = {'categories': [], 'city': None, 'country': None, 'architects': [], 'area': None, 'year': None, 'description': None}
        title_tag = soup.find(class_='afd-title-big--bmargin-big')
        project_title = title_tag.get_text(strip=True).split('/')[0].strip() if title_tag else "Untitled Project"

        category_container = soup.find(class_='afd-specs__header-category')
        if category_container: data['categories'] = [a.text.strip() for a in category_container.find_all('a')]

        location_container = soup.find(class_='afd-specs__header-location')
        if location_container:
            location_parts = location_container.get_text(strip=True).split(',')
            if location_parts: data['city'] = location_parts[0].strip()
            country_link = location_container.find('a')
            if country_link: data['country'] = country_link.text.strip()

        for item in soup.select('.afd-specs__item'):
            key_tag = item.find(class_='afd-specs__key')
            value_tag = item.find(class_='afd-specs__value')
            if not key_tag or not value_tag: continue
            key_text = key_tag.get_text(strip=True)
            if 'Architects' in key_text: data['architects'] = [a.text.strip() for a in value_tag.find_all('a')]
            elif 'Area' in key_text: data['area'] = value_tag.text.replace('m²', '').strip()
            elif 'Year' in key_text: data['year'] = value_tag.text.strip()
            # Add more fields here if needed (e.g., Manufacturers, etc.)

        # Description extraction
        description_paragraphs = []
        desc_section = soup.find('div', class_='the-content') # Target the main content div
        if desc_section:
            for paragraph in desc_section.find_all('p', recursive=False): # Find direct children paragraphs
                extracted_text = extract_text_with_spacing(paragraph)
                normalized_text = re.sub(r'\s+', ' ', extracted_text).strip()
                if normalized_text: # Avoid empty strings
                     description_paragraphs.append(normalized_text)
        else: # Fallback if specific content div isn't found
             for paragraph in soup.find_all('p'):
                 extracted_text = extract_text_with_spacing(paragraph)
                 normalized_text = re.sub(r'\s+', ' ', extracted_text).strip()
                 if normalized_text:
                     description_paragraphs.append(normalized_text)

        data['description'] = purge_description(description_paragraphs) # Clean the description list
        # --- End Data Extraction ---

        # Construct final dict
        project_dict = {
            'Project ID': project_id,
            'Project Title': project_title,
            'Categories': data['categories'] or None,
            'City': data['city'],
            'Country': data['country'],
            'Architects': data['architects'] or None,
            'Area': f"{data['area']} m²" if data['area'] else None,
            'Year': data['year'],
            'Project URL': project_link,
            'Description': data['description'] or None
        }
        project_dict = {k: v for k, v in project_dict.items() if v} # Remove keys with None/empty values

        # --- Save details JSON ---
        details_filename = f'{project_id}_details.json'
        # Ensure the base directory exists (needed if images aren't downloaded first)
        Path(project_id).mkdir(parents=True, exist_ok=True)
        details_filepath = Path(project_id) / details_filename

        with open(details_filepath, 'w', encoding='utf-8') as f:
            json.dump(project_dict, f, ensure_ascii=False, indent=2)

        print(f'Project details saved to: {details_filepath}')
        return "downloaded" # Use status consistent with original script

    except Exception as e:
        print(f"[ERROR] Error processing details for {project_link}: {e}")
        traceback.print_exc()
        return "error"


# --- scrape_gallery_thumbnails function (MODIFIED: Accepts driver) ---
def scrape_gallery_thumbnails(driver: WebDriver, url: str) -> Tuple[bool, List[Dict[str, str]]]:
    """
    Scrapes gallery thumbnail information (URL and title) using a provided WebDriver.
    """
    gallery_items: List[Dict[str, str]] = []
    # WebDriver is now passed in, no internal creation/destruction

    try:
        print(f"Fetching project page with Selenium: {url}")
        driver.get(url)

        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "gallery-thumbs"))
            )
            print("Gallery thumbnails section loaded.")
        except TimeoutException:
            print("Timeout waiting for gallery thumbnails section.")
            # Consider if this should be a hard fail
            pass # Attempt to parse anyway

        # Use the passed driver to get page source
        initial_page_source = driver.page_source
        soup = BeautifulSoup(initial_page_source, "html.parser")

        thumbnail_items = soup.find_all("li", class_="gallery-thumbs-item")
        if not thumbnail_items:
             print("Could not find any 'gallery-thumbs-item' list items.")
             # It might be valid for a project to have no images/thumbnails
             return True, gallery_items # Return True but empty list

        print(f"Found {len(thumbnail_items)} potential thumbnail items.")
        processed_count = 0
        for i, li in enumerate(thumbnail_items):
            try:
                link = li.find("a", class_="gallery-thumbs-link")
                if not link: continue
                href = link.get("href")
                if not href: continue

                full_href = urljoin(url, href)
                title = link.get("title", "").strip()
                if not title:
                    img = link.find("img")
                    if img: title = img.get("alt", "").strip()
                title = title if title else f"Untitled_{i+1}"

                if not full_href or full_href == url: continue # Skip self-links

                item = {"href": full_href, "title": title}
                gallery_items.append(item)
                processed_count += 1
            except Exception as e:
                print(f"  Error processing thumbnail item {i+1}: {str(e)}")
                continue # Skip this item

        print(f"Gallery thumbnail scraping complete. Successfully extracted {processed_count} items.")
        # Success is true if the process ran, even if 0 items were found/processed
        return True, gallery_items

    except WebDriverException as e:
        print(f"[ERROR] WebDriver error during thumbnail scraping for {url}: {e}")
        return False, gallery_items
    except Exception as e:
        print(f"[ERROR] Unexpected error in scrape_gallery_thumbnails for {url}: {e}")
        traceback.print_exc()
        return False, gallery_items
    # No driver.quit() here - managed globally


# --- download_gallery_image function (Unchanged) ---
def download_gallery_image(page_url: str, save_directory: Path, base_filename: str) -> Tuple[bool, Optional[List[str]], Optional[str], Optional[str]]:
    # --- (Content of this function remains the same as in the previous version) ---
    # ... (Keep the function body exactly as provided in the previous step) ...
    # ... (It uses requests, parses JSON, downloads image, returns tuple) ...
    success = False
    tags = None
    caption = None
    target_image_info = None
    final_filename = None

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        logging.info(f"Fetching image page HTML: {page_url}")
        response = requests.get(page_url, headers=headers, timeout=15)
        response.raise_for_status()
        logging.info("HTML fetched.")

        soup = BeautifulSoup(response.text, 'html.parser')
        gallery_div = soup.find('div', id='gallery-items')
        if not gallery_div:
            logging.error("No gallery container ('div#gallery-items').")
            return success, tags, caption, final_filename

        data_images_json = gallery_div.get('data-images')
        default_image_id_fragment = gallery_div.get('data-id')
        if not data_images_json:
            logging.error("No 'data-images' attribute.")
            return success, tags, caption, final_filename

        try:
            image_data = json.loads(data_images_json)
            if not isinstance(image_data, list) or not image_data:
                 logging.error("'data-images' not a valid list.")
                 return success, tags, caption, final_filename
        except json.JSONDecodeError as e:
            logging.error(f"JSON parse error: {e}")
            return success, tags, caption, final_filename

        parsed_url = urlparse(page_url)
        url_fragment = parsed_url.fragment
        target_image_info = None

        if url_fragment:
            target_id = url_fragment.split('-')[0]
            for img_info in image_data:
                if 'link' in img_info and target_id in img_info['link']:
                     target_image_info = img_info
                     break
            if not target_image_info:
                 logging.warning(f"Fragment '{target_id}' not matched.")

        if not target_image_info: # Fallback
            if default_image_id_fragment:
                for img_info in image_data:
                     if 'link' in img_info and default_image_id_fragment in img_info['link']:
                         target_image_info = img_info
                         break
                if not target_image_info:
                     logging.warning(f"Default ID {default_image_id_fragment} not matched.")
                     if image_data: target_image_info = image_data[0] # Use first
            else:
                logging.warning("No fragment/default ID. Using first image.")
                if image_data: target_image_info = image_data[0] # Use first

        if not target_image_info:
             logging.error("Cannot determine target image.")
             return success, tags, caption, final_filename

        image_url = target_image_info.get('url_large') or target_image_info.get('url_slideshow')
        caption = target_image_info.get('caption', 'No Caption')
        raw_tags = target_image_info.get('tags', [])
        tags = [tag.get('name', '').strip() for tag in raw_tags if isinstance(tag, dict) and 'name' in tag]

        if not image_url:
            logging.error(f"Target image missing URL (Link: {target_image_info.get('link')}).")
            return success, tags, caption, final_filename

        parsed_image_url = urlparse(image_url)
        image_path_part = Path(unquote(parsed_image_url.path))
        extension = image_path_part.suffix
        if not extension or len(extension) > 5:
             extension = ".jpg"
             logging.warning(f"Using default .jpg for {image_url}")

        final_filename = f"{base_filename}{extension}"
        save_path = save_directory / final_filename
        logging.info(f"Target save path: {save_path}")

        logging.info(f"Downloading image: {image_url}...")
        img_response = requests.get(image_url, stream=True, headers=headers, timeout=30)
        img_response.raise_for_status()

        with open(save_path, 'wb') as f:
            for chunk in img_response.iter_content(chunk_size=8192):
                f.write(chunk)

        logging.info(f"Image download SUCCESS.")
        success = True

    except requests.exceptions.Timeout: logging.error("Request timed out.")
    except requests.exceptions.HTTPError as e: logging.error(f"HTTP Error: {e.response.status_code}")
    except requests.exceptions.RequestException as e: logging.error(f"Network error: {e}")
    except IOError as e: logging.error(f"File IO error: {e}")
    except Exception as e: logging.error(f"Unexpected error: {e}", exc_info=False)

    finally:
        return success, tags, caption, final_filename


# --- Main Image Processing Function (MODIFIED: Accepts driver, saves images JSON) ---
def process_project_images(driver: WebDriver, project_url: str) -> Tuple[bool, Optional[Path]]:
    """
    Processes images for a single ArchDaily project using a provided WebDriver:
    scrapes thumbnails, downloads images, saves image metadata JSON.

    Args:
        driver (WebDriver): The shared Selenium WebDriver instance.
        project_url (str): The URL of the ArchDaily project page.

    Returns:
        Tuple[bool, Optional[Path]]: Overall success status for image processing,
                                     and the Path object to the project folder if successful.
    """
    print(f"--- Processing Project Images for: {project_url} ---")
    overall_image_success = False # Track if at least one image downloads

    # --- Extract Project ID and Set Up Base Folder ---
    project_id = None
    try:
        path_parts = [part for part in urlparse(project_url).path.strip('/').split('/') if part]
        for part in path_parts:
            if part.isdigit():
                project_id = part
                break
    except Exception as e:
        print(f"[ERROR] Error extracting project ID for images: {e}")
        return overall_image_success, None

    if not project_id:
        print(f"[ERROR] Could not extract project ID for images: {project_url}.")
        return overall_image_success, None

    base_download_folder = Path(project_id)
    print(f"Image Project ID: {project_id}")
    print(f"Image download directory: '{base_download_folder.resolve()}'")

    try:
        base_download_folder.mkdir(parents=True, exist_ok=True)
    except OSError as e:
        print(f"[ERROR] Error creating/accessing image directory '{base_download_folder}': {e}.")
        return overall_image_success, None

    # --- Scrape Thumbnails using shared driver ---
    print(f"Scraping thumbnails...")
    # Pass the shared driver to the modified scrape_gallery_thumbnails
    success_scrape, gallery_items = scrape_gallery_thumbnails(driver, project_url)

    if not success_scrape:
        print("Thumbnail scraping failed. Cannot proceed with image downloads.")
        return overall_image_success, base_download_folder # Return folder path but False status
    if not gallery_items:
        print("No gallery items found to download.")
        # Consider this a success in terms of processing, just nothing to download
        return True, base_download_folder

    # --- Iterate and Download ---
    print(f"--- Starting image downloads for {len(gallery_items)} items ---")
    all_metadata = []
    successful_downloads = 0
    for index, item in enumerate(gallery_items):
        item_url = item.get('href')
        item_title = item.get('title', f'image_{index+1}')

        print(f"\n>>> Downloading Item {index+1}/{len(gallery_items)}: '{item_title}' <<<")

        if not item_url:
            print("    Skipping: Missing URL.")
            continue

        base_filename = f"{project_id}_{index+1:02d}"

        success_download, tags, caption, final_filename = download_gallery_image(
            item_url,
            base_download_folder,
            base_filename
        )

        if success_download and final_filename:
            print(f"    SUCCESS: Saved as {final_filename}")
            metadata_entry = {
                "filename": final_filename,
                "tags": tags if tags else [],
                "caption": caption if caption else ""
            }
            all_metadata.append(metadata_entry)
            successful_downloads += 1
            overall_image_success = True # Mark overall success if at least one downloads
        else:
            print(f"    FAILED")

        time.sleep(0.5) # Delay

    print(f"\n--- Image download process complete: {successful_downloads}/{len(gallery_items)} successful. ---")

    # --- Save Image Metadata JSON ---
    if all_metadata:
        metadata_filepath = base_download_folder / f"{project_id}_images.json" # New filename
        print(f"Saving image metadata to {metadata_filepath}")
        try:
            with open(metadata_filepath, 'w', encoding='utf-8') as f:
                json.dump(all_metadata, f, indent=4, ensure_ascii=False)
            print("Image metadata saved.")
        except IOError as e:
            print(f"[ERROR] Failed to save image metadata JSON: {e}")
    else:
        print("No image metadata collected.")

    print(f"--- Finished processing images for project {project_id} ---")
    # Return overall success status and the folder path
    return overall_image_success, base_download_folder


# --- CSV Handling Functions (Fixed typo) ---
def update_csv_status(csv_file, project_code, status):
    rows = []
    try:
        with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            header = next(reader)
            try:
                 status_index = header.index('status')
            except ValueError:
                 print(f"[ERROR] 'status' column not found in CSV header: {header}")
                 return # Cannot update status
            rows.append(header)
            for row in reader:
                if row and row[0] == project_code: # Check if row is not empty
                    # Ensure row has enough columns before accessing status_index
                    if len(row) > status_index:
                        row[status_index] = status
                    else:
                        # Handle rows that might be shorter than the header
                        print(f"[WARN] Row for {project_code} is too short, cannot update status.")
                        # Optionally append empty strings to match header length
                        row.extend([''] * (len(header) - len(row)))
                        row[status_index] = status # Try updating now
                rows.append(row)
    except FileNotFoundError:
        print(f"[ERROR] CSV file not found: {csv_file}")
        return
    except Exception as e:
        print(f"[ERROR] Error reading CSV file {csv_file}: {e}")
        return

    try:
        # Use temporary file for safer writing
        temp_file = Path(csv_file).with_suffix('.tmp')
        with open(temp_file, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(rows)
        # Replace original file with temporary file
        os.replace(temp_file, csv_file)
        print(f"Updated status for {project_code} to '{status}' in {csv_file}")
    except Exception as e:
        print(f"[ERROR] Error writing updated CSV file {csv_file}: {e}")
        if temp_file.exists():
             os.remove(temp_file) # Clean up temp file on error


def remove_csv_status(csv_file):
    # remove all the status in the csv file
    rows = []
    try:
        with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            header = next(reader)
            try:
                 status_index = header.index('status')
            except ValueError:
                 print(f"[ERROR] 'status' column not found in CSV header: {header}")
                 return
            rows.append(header)
            for row in reader:
                 if row: # Check if row is not empty
                      if len(row) > status_index:
                           row[status_index] = "" # Clear the status
                      else:
                           # Handle short rows if necessary, or just skip status update
                           pass
                 rows.append(row) # Append row even if it was short/empty
    except FileNotFoundError:
        print(f"[ERROR] CSV file not found: {csv_file}")
        return
    except Exception as e:
        print(f"[ERROR] Error reading CSV file {csv_file} for status removal: {e}")
        return

    try:
        # Use temporary file for safer writing
        temp_file = Path(csv_file).with_suffix('.tmp')
        with open(temp_file, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(rows)
        os.replace(temp_file, csv_file)
        print(f"Cleared all statuses in {csv_file}")
    except Exception as e:
        print(f"[ERROR] Error writing CSV file {csv_file} after clearing statuses: {e}")
        if temp_file.exists():
             os.remove(temp_file)


# --- Main Execution Logic (Integrated) ---
if __name__ == "__main__":
    csv_file = './archdaily_projects.csv'
    base_url = 'https://www.archdaily.com' # Define base_url needed for link construction

    # --- Initialize Shared WebDriver ---
    driver = None
    try:
        print("Initializing Shared WebDriver for the session...")
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--log-level=3')
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        os.environ['WDM_LOG_LEVEL'] = '0'

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("Shared WebDriver Initialized.")

        # --- Optional: Reset all statuses before starting ---
        # print(f"\nClearing previous statuses in {csv_file}...")
        # remove_csv_status(csv_file)
        # print("Statuses cleared.")
        # --- End Optional Reset ---


        # --- Read CSV and Process Projects ---
        print(f"\nReading projects from {csv_file}...")
        try:
            with open(csv_file, 'r', encoding='utf-8') as file:
                reader = csv.reader(file)
                header = next(reader)
                try:
                    status_index = header.index('status')
                    # Assuming project code is the first column
                    code_index = 0
                except ValueError:
                    print("[ERROR] CSV file must contain 'status' column. Exiting.")
                    exit()

                rows = list(reader) # Read all rows into memory for easier processing

        except FileNotFoundError:
            print(f"[ERROR] CSV file not found: {csv_file}. Exiting.")
            exit()
        except Exception as e:
            print(f"[ERROR] Failed to read CSV file {csv_file}: {e}. Exiting.")
            exit()

        print(f"Found {len(rows)} projects in CSV.")

        for i, row in enumerate(rows):
            if not row: # Skip empty rows
                 print(f"Skipping empty row {i+1}")
                 continue
            try:
                project_code = row[code_index]
                status = row[status_index] if len(row) > status_index else ""
            except IndexError:
                 print(f"[WARN] Skipping row {i+1} due to insufficient columns: {row}")
                 continue

            print(f"\n{'='*10} Processing Project {i+1}/{len(rows)}: {project_code} {'='*10}")

            # Skip based on status
            skip_statuses = ['downloaded', 'error', 'incomplete', 'duplicate', 'delete'] # Add more if needed
            if status.lower() in skip_statuses:
                print(f"Skipping project {project_code} (Status: '{status}').")
                continue

            # Construct full project link
            # Ensure project_code doesn't have leading/trailing slashes if base_url ends with one
            project_link = f"{base_url.strip('/')}/{project_code.strip('/')}"
            print(f"Project URL: {project_link}")

            # --- Step 1: Scrape Project Details ---
            scraper_status = project_scraper(project_link) # Uses Requests

            # --- Step 2: Process Project Images (if details scraped successfully) ---
            downloader_overall_success = False
            if scraper_status == "downloaded":
                # Pass the shared driver here
                downloader_overall_success, _ = process_project_images(driver, project_link)
            else:
                print("Skipping image download because project details scraping failed.")

            # --- Step 3: Determine Final Status and Update CSV ---
            final_status = "error" # Default to error
            if scraper_status == "downloaded":
                if downloader_overall_success:
                    final_status = "downloaded" # Both parts succeeded
                else:
                    # Scraper OK, but downloader failed or found no images to download successfully
                    final_status = "incomplete"
            # Keep "error" if scraper_status was "error"

            print(f"Final status for {project_code}: {final_status}")
            update_csv_status(csv_file, project_code, final_status)

            # Optional short delay between projects
            time.sleep(1)

        print("\nFinished processing all projects from CSV.")

    except Exception as e:
        print(f"\nAn unexpected error occurred during the main processing loop: {e}")
        traceback.print_exc()
    finally:
        # --- Clean Up Shared WebDriver ---
        if driver:
            print("\nQuitting Shared WebDriver...")
            try:
                driver.quit()
                print("Shared WebDriver quit successfully.")
            except Exception as e:
                print(f"Error quitting Shared WebDriver: {e}")

2025-04-07 00:19:46,984 - INFO - ====== WebDriver manager ======


Initializing Shared WebDriver for the session...


2025-04-07 00:19:47,776 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-07 00:19:48,130 - INFO - Get LATEST chromedriver version for google-chrome
2025-04-07 00:19:48,544 - INFO - Driver [C:\Users\15717\.wdm\drivers\chromedriver\win64\134.0.6998.165\chromedriver-win32/chromedriver.exe] found in cache


Shared WebDriver Initialized.

Reading projects from ./archdaily_projects.csv...
Found 8 projects in CSV.

========== Processing Project 1/8: 904658 ==========
Project URL: https://www.archdaily.com/904658
--- Scraping Project Details for: https://www.archdaily.com/904658 ---
Project details HTML parsed.
Project details saved to: 904658\904658_details.json
--- Processing Project Images for: https://www.archdaily.com/904658 ---
Image Project ID: 904658
Image download directory: 'D:\999_RAG\crawler\904658'
Scraping thumbnails...
Fetching project page with Selenium: https://www.archdaily.com/904658


2025-04-07 00:20:14,614 - INFO - Fetching image page HTML: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1acc7f197cc96d000033d-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo


...

    SUCCESS: Saved as 904658_14.jpg


2025-04-07 00:21:07,767 - INFO - Fetching image page HTML: https://www.archdaily.com/904658/phillip-and-patricia-frost-museum-of-science-grimshaw-architects/5bd1ab87f197cc96d0000334-phillip-and-patricia-frost-museum-of-science-grimshaw-architects-photo



>>> Downloading Item 15/24: 'Phillip and Patricia Frost Museum of Science / Grimshaw Architects' <<<


2025-04-07 00:21:09,450 - INFO - HTML fetched.
2025-04-07 00:21:09,462 - INFO - Target save path: 904658\904658_15.jpg
2025-04-07 00:21:09,462 - INFO - Downloading image: https://images.adsttc.com/media/images/5bd1/ab87/f197/cc96/d000/0334/large_jpg/Grimshaw_Frost_2149.jpg?1540467586...
2025-04-07 00:21:11,056 - INFO - Image download SUCCESS.


    SUCCESS: Saved as 904658_15.jpg


In [3]:
import csv
import os
import requests
import time
import re
import json
from urllib.parse import urlparse, unquote, urljoin
from pathlib import Path
from bs4 import BeautifulSoup
import logging
from typing import List, Dict, Tuple, Optional, Any
import concurrent.futures
import threading # For thread-local data if needed, or getting thread ID

# --- Selenium Imports ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver # Import WebDriver type hint
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import traceback # For detailed error printing

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(threadName)s - %(levelname)s - %(message)s') # Added threadName
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("selenium").setLevel(logging.WARNING)
logging.getLogger("webdriver_manager").setLevel(logging.WARNING)


# --- Helper Functions for project_scraper (Unchanged) ---
def extract_text_with_spacing(element):
    """Recursively extracts text preserving approximate spacing."""
    result = ""
    if element is None:
        return result
    for content in element.contents:
        if hasattr(content, 'name') and content.name: # if it's a tag
            result += extract_text_with_spacing(content)
        elif hasattr(content, 'string'): # if it's NavigableString (text)
            result += content.string
        # Handle cases where content might be None or other types
    return result

def purge_description(description_list):
    """Cleans the list of description paragraphs."""
    # Remove items with 3 or fewer words
    description_list = [x for x in description_list if len(x.split()) > 3]
    # Remove specific boilerplate text
    description_list = [x for x in description_list if "You'll now receive updates based on what you follow!" not in x]
    # Remove "Check the" items
    description_list = [x for x in description_list if not x.startswith("Check the")]
    # Remove "Save this picture!" items
    description_list = [x for x in description_list if "Save this picture!" not in x]
    # Remove duplicates while preserving order
    seen = set()
    description_list = [x for x in description_list if not (x in seen or seen.add(x))]
    return description_list


# --- project_scraper function (Unchanged - uses requests) ---
def project_scraper(project_link: str) -> Tuple[str, Optional[str]]:
    """
    Scrapes project details (metadata like architects, area, year, description)
    and saves them to [project_id]_details.json.
    Uses Requests, not Selenium.

    Args:
        project_link (str): The URL of the ArchDaily project page.

    Returns:
        Tuple[str, Optional[str]]: Status ("downloaded" or "error") and the project_id if successful.
    """
    logging.info(f"--- Scraping Project Details for: {project_link} ---")
    project_id = None # Initialize project_id
    headers = { # Add headers for requests
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9'
    }
    try:
        response = requests.get(project_link, headers=headers, timeout=20)
        response.raise_for_status() # Check for HTTP errors
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error fetching details for {project_link}: {e}")
        return "error", None

    try:
        soup = BeautifulSoup(response.content, 'html.parser')
        logging.debug('Project details HTML parsed.') # Changed to debug

        # --- Extract project ID ---
        try:
            path_parts = [part for part in urlparse(project_link).path.strip('/').split('/') if part]
            for part in path_parts:
                if part.isdigit():
                    project_id = part
                    break
        except Exception:
            pass # Ignore if ID extraction fails here, use fallback

        if not project_id:
            project_id_match = re.search(r'/(\d+)/', project_link)
            if project_id_match:
                project_id = project_id_match.group(1)
            else:
                logging.warning(f"Could not determine project ID for filename from {project_link}.")
                # Try finding it in the body as a last resort
                script_tag = soup.find('script', string=re.compile(r'window\.classifiedId'))
                if script_tag:
                    match = re.search(r'window\.classifiedId\s*=\s*(\d+);', script_tag.string)
                    if match:
                        project_id = match.group(1)
                        logging.info(f"Found project ID {project_id} from script tag.")

        if not project_id:
            logging.error(f"CRITICAL: Failed to determine project ID for {project_link}. Cannot save details.")
            return "error", None # Return error if ID is essential and not found

        # --- Data Extraction Logic ---
        data = {'categories': [], 'city': None, 'country': None, 'architects': [], 'area': None, 'year': None, 'description': None}
        title_tag = soup.find(class_='afd-title-big--bmargin-big')
        project_title = title_tag.get_text(strip=True).split('/')[0].strip() if title_tag else f"Project {project_id}"

        category_container = soup.find(class_='afd-specs__header-category')
        if category_container: data['categories'] = [a.text.strip() for a in category_container.find_all('a')]

        location_container = soup.find(class_='afd-specs__header-location')
        if location_container:
            location_parts = location_container.get_text(strip=True).split(',')
            if location_parts: data['city'] = location_parts[0].strip()
            country_link = location_container.find('a')
            if country_link: data['country'] = country_link.text.strip()

        for item in soup.select('.afd-specs__item'):
            key_tag = item.find(class_='afd-specs__key')
            value_tag = item.find(class_='afd-specs__value')
            if not key_tag or not value_tag: continue
            key_text = key_tag.get_text(strip=True)
            if 'Architects' in key_text: data['architects'] = [a.text.strip() for a in value_tag.find_all('a')]
            elif 'Area' in key_text: data['area'] = value_tag.text.replace('m²', '').strip()
            elif 'Year' in key_text: data['year'] = value_tag.text.strip()
            # Add more fields here if needed

        # Description extraction
        description_paragraphs = []
        desc_section = soup.find('div', class_='the-content')
        if desc_section:
            for paragraph in desc_section.find_all('p', recursive=False):
                extracted_text = extract_text_with_spacing(paragraph)
                normalized_text = re.sub(r'\s+', ' ', extracted_text).strip()
                if normalized_text:
                    description_paragraphs.append(normalized_text)
        else:
            logging.warning(f"Could not find specific 'the-content' div for description on {project_link}. Falling back to all <p> tags.")
            for paragraph in soup.find_all('p'):
                extracted_text = extract_text_with_spacing(paragraph)
                normalized_text = re.sub(r'\s+', ' ', extracted_text).strip()
                if normalized_text:
                    description_paragraphs.append(normalized_text)

        data['description'] = purge_description(description_paragraphs)

        # --- Construct final dict ---
        project_dict = {
            'Project ID': project_id,
            'Project Title': project_title,
            'Categories': data['categories'] or None,
            'City': data['city'],
            'Country': data['country'],
            'Architects': data['architects'] or None,
            'Area': f"{data['area']} m²" if data['area'] else None,
            'Year': data['year'],
            'Project URL': project_link,
            'Description': data['description'] or None
        }
        project_dict = {k: v for k, v in project_dict.items() if v is not None and v != []} # Remove keys with None/empty list values

        # --- Save details JSON ---
        details_filename = f'{project_id}_details.json'
        # Ensure the base directory exists
        project_dir = Path(project_id)
        project_dir.mkdir(parents=True, exist_ok=True)
        details_filepath = project_dir / details_filename

        with open(details_filepath, 'w', encoding='utf-8') as f:
            json.dump(project_dict, f, ensure_ascii=False, indent=2)

        logging.info(f'Project details saved to: {details_filepath}')
        return "downloaded", project_id # Return success and project_id

    except Exception as e:
        logging.error(f"Error processing details for {project_link}: {e}")
        # traceback.print_exc() # Keep this for debugging if needed
        return "error", project_id # Return error, but still return ID if extracted


# --- WebDriver Creation Helper ---
def create_webdriver() -> Optional[WebDriver]:
    """Creates and returns a Selenium WebDriver instance."""
    driver = None
    try:
        # logging.info("Initializing WebDriver for thread...") # Can be noisy
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--log-level=3') # Reduce browser console noise
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        options.add_argument("--blink-settings=imagesEnabled=false") # Optionally disable images in Selenium for speed
        os.environ['WDM_LOG_LEVEL'] = '0' # Suppress webdriver-manager logs

        # Use a context manager for service if possible, or ensure it's managed
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        # logging.info("WebDriver Initialized for thread.")
        return driver
    except WebDriverException as e:
        logging.error(f"Failed to initialize WebDriver: {e}")
        if driver:
            driver.quit()
        return None
    except Exception as e:
        logging.error(f"Unexpected error creating WebDriver: {e}")
        if driver:
            driver.quit()
        return None

# --- scrape_gallery_thumbnails function (MODIFIED: Accepts driver, more robust waits) ---
def scrape_gallery_thumbnails(driver: WebDriver, url: str) -> Tuple[bool, List[Dict[str, str]]]:
    """
    Scrapes gallery thumbnail information (URL and title) using a provided WebDriver.
    """
    gallery_items: List[Dict[str, str]] = []
    try:
        logging.info(f"Fetching project page with Selenium for thumbnails: {url}")
        driver.get(url)

        try:
            # Wait for EITHER the gallery OR a known 'no images' element OR the main content area
            wait = WebDriverWait(driver, 20)
            wait.until(
                EC.any_of(
                    EC.presence_of_element_located((By.CLASS_NAME, "gallery-thumbs")),
                    EC.presence_of_element_located((By.ID, "content")), # General content area as fallback
                    EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'No images found')]")) # Example, adjust if site has specific message
                )
            )
            logging.debug("Relevant section loaded (gallery or content).")

            # Check if the gallery specifically exists
            try:
                driver.find_element(By.CLASS_NAME, "gallery-thumbs")
                logging.debug("Gallery thumbnails section found.")
            except NoSuchElementException:
                logging.info("No 'gallery-thumbs' section found on the page. Assuming no images.")
                return True, [] # No gallery is a valid outcome

        except TimeoutException:
            logging.warning("Timeout waiting for gallery thumbnails section or page content. Proceeding with parse attempt.")
            # Allow parsing attempt even on timeout

        # Use the passed driver to get page source
        page_source = driver.page_source # Get source *after* waiting
        soup = BeautifulSoup(page_source, "html.parser")

        thumbnail_items = soup.find_all("li", class_="gallery-thumbs-item")
        if not thumbnail_items:
            logging.info("Could not find any 'gallery-thumbs-item' list items after parsing.")
            return True, gallery_items # Return True but empty list

        logging.info(f"Found {len(thumbnail_items)} potential thumbnail items.")
        processed_count = 0
        for i, li in enumerate(thumbnail_items):
            try:
                link = li.find("a", class_="gallery-thumbs-link")
                if not link: continue
                href = link.get("href")
                if not href: continue

                # Make URL absolute
                full_href = urljoin(url, href)

                title = link.get("title", "").strip()
                if not title:
                    img = link.find("img")
                    if img: title = img.get("alt", "").strip()
                # Sanitize title slightly for filename use later if needed
                sanitized_title = re.sub(r'[<>:"/\\|?*]', '_', title) # Basic sanitization
                title = sanitized_title if sanitized_title else f"Untitled_{i+1}"

                if not full_href or full_href == url or '#' not in full_href: # Ensure it's a link to an image within the gallery (often uses fragments)
                     logging.debug(f"Skipping non-gallery or self-link: {full_href}")
                     continue

                item = {"href": full_href, "title": title}
                gallery_items.append(item)
                processed_count += 1
            except Exception as e:
                logging.warning(f"Error processing thumbnail item {i+1}: {str(e)}")
                continue # Skip this item

        logging.info(f"Gallery thumbnail scraping complete. Successfully extracted {processed_count} items.")
        return True, gallery_items

    except WebDriverException as e:
        logging.error(f"WebDriver error during thumbnail scraping for {url}: {e}")
        return False, gallery_items
    except Exception as e:
        logging.error(f"Unexpected error in scrape_gallery_thumbnails for {url}: {e}")
        # traceback.print_exc()
        return False, gallery_items
    # No driver.quit() here - managed by the caller (process_project_images)


# --- download_gallery_image function (Unchanged - uses requests) ---
def download_gallery_image(page_url: str, save_directory: Path, base_filename: str) -> Tuple[bool, Optional[List[str]], Optional[str], Optional[str]]:
    """Downloads a single image and extracts metadata using requests."""
    # --- (Content of this function remains the same as in the original prompt) ---
    # ... (It uses requests, parses JSON, downloads image, returns tuple) ...
    success = False
    tags = None
    caption = None
    final_filename = None

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Referer': page_url # Often helpful
    }

    try:
        logging.debug(f"Fetching image page HTML: {page_url}")
        response = requests.get(page_url, headers=headers, timeout=20) # Increased timeout slightly
        response.raise_for_status()
        logging.debug("HTML fetched for image details.")

        soup = BeautifulSoup(response.text, 'html.parser')
        gallery_div = soup.find('div', id='gallery-items')
        if not gallery_div:
            logging.error(f"No gallery container ('div#gallery-items') found on {page_url}.")
            # Attempt fallback: look for data-images on body or another likely element
            script_tag = soup.find('script', string=re.compile(r'window\.__INITIAL_PROPS__'))
            if script_tag:
                 logging.debug("Attempting to extract image data from __INITIAL_PROPS__ script")
                 try:
                    # This requires inspecting the specific structure of __INITIAL_PROPS__
                    # It's often a large JSON blob. This is a placeholder guess.
                    json_str = script_tag.string.partition('=')[2].strip().rstrip(';')
                    initial_props = json.loads(json_str)
                    # Navigate the structure - this WILL need adjustment based on actual site structure
                    image_data = initial_props.get('gallery', {}).get('images', []) # EXAMPLE PATH
                    if image_data:
                        logging.info("Successfully extracted image data from __INITIAL_PROPS__.")
                    else:
                         logging.error("Found __INITIAL_PROPS__ but failed to extract image data structure.")
                         return success, tags, caption, final_filename
                 except (json.JSONDecodeError, AttributeError, KeyError, IndexError) as e:
                     logging.error(f"Error parsing __INITIAL_PROPS__: {e}")
                     return success, tags, caption, final_filename
            else:
                logging.error(f"Fallback failed: No 'div#gallery-items' or '__INITIAL_PROPS__' script found.")
                return success, tags, caption, final_filename
        else:
            # Original method if gallery_div is found
            data_images_json = gallery_div.get('data-images')
            default_image_id_fragment = gallery_div.get('data-id') # Might be just the ID, not fragment
            if not data_images_json:
                logging.error("No 'data-images' attribute found on gallery div.")
                return success, tags, caption, final_filename

            try:
                image_data = json.loads(data_images_json)
                if not isinstance(image_data, list) or not image_data:
                    logging.error(f"'data-images' is not a valid list or is empty on {page_url}.")
                    return success, tags, caption, final_filename
            except json.JSONDecodeError as e:
                logging.error(f"JSON parse error for 'data-images' on {page_url}: {e}")
                return success, tags, caption, final_filename

        # --- Image Selection Logic (using URL fragment) ---
        parsed_url = urlparse(page_url)
        url_fragment = parsed_url.fragment # e.g., '5f1f...-image'
        target_image_info = None

        # Try matching based on the URL fragment
        if url_fragment:
            # Extract the ID part before any potential suffix like '-image'
            target_id = url_fragment.split('-')[0]
            logging.debug(f"Looking for image matching fragment ID: {target_id}")
            for img_info in image_data:
                # Check against 'id' or a relevant part of 'link' or another identifier field
                # Adjust this check based on the actual structure within image_data
                img_id_str = str(img_info.get('id', '')) # Example: Check 'id' field
                img_link = img_info.get('link', '') # Example: Check 'link' field
                if target_id == img_id_str or (img_link and target_id in img_link):
                    target_image_info = img_info
                    logging.debug(f"Found match via fragment: ID={img_id_str}")
                    break
            if not target_image_info:
                 logging.warning(f"URL fragment '{url_fragment}' (target ID '{target_id}') provided, but no matching image found in data.")

        # Fallback: If no fragment match or no fragment provided, try using the *first* image
        if not target_image_info:
            if image_data:
                logging.debug("No fragment match or no fragment. Using first image in the data list.")
                target_image_info = image_data[0]
            else:
                 logging.error("Cannot determine target image: No fragment match and image data list is empty.")
                 return success, tags, caption, final_filename # Should have been caught earlier, but double-check

        # --- Extract Details from Selected Image Info ---
        image_url = target_image_info.get('url_large') or target_image_info.get('url_slideshow') # Prioritize larger
        # Clean caption: Remove HTML tags if present
        raw_caption = target_image_info.get('caption', '')
        caption = BeautifulSoup(raw_caption, "html.parser").get_text(separator=' ', strip=True) if raw_caption else "No Caption"

        # Extract tags safely
        raw_tags = target_image_info.get('tags', [])
        tags = []
        if isinstance(raw_tags, list):
            for tag in raw_tags:
                if isinstance(tag, dict) and 'name' in tag:
                    tag_name = tag.get('name', '').strip()
                    if tag_name:
                        tags.append(tag_name)
        else:
            logging.warning(f"Unexpected format for tags: {raw_tags}")


        if not image_url:
            img_id_for_error = target_image_info.get('id', target_image_info.get('link', 'Unknown Identifier'))
            logging.error(f"Target image info found (ID/Link: {img_id_for_error}), but it's missing 'url_large' or 'url_slideshow'.")
            return success, tags, caption, final_filename # Return metadata found so far, but fail download

        # --- Download Image ---
        try:
            parsed_image_url = urlparse(image_url)
            image_path_part = Path(unquote(parsed_image_url.path))
            extension = image_path_part.suffix.lower()
            # Basic validation for common image extensions
            if not extension or extension not in ['.jpg', '.jpeg', '.png', '.gif', '.webp']:
                # Try getting extension from Content-Type header as a fallback
                try:
                    head_response = requests.head(image_url, headers=headers, timeout=10, allow_redirects=True)
                    head_response.raise_for_status()
                    content_type = head_response.headers.get('Content-Type', '').lower()
                    if 'jpeg' in content_type or 'jpg' in content_type: extension = '.jpg'
                    elif 'png' in content_type: extension = '.png'
                    elif 'gif' in content_type: extension = '.gif'
                    elif 'webp' in content_type: extension = '.webp'
                    else: extension = ".jpg" # Final fallback
                    logging.warning(f"Guessed extension '{extension}' from Content-Type for {image_url}")
                except requests.exceptions.RequestException:
                     logging.warning(f"Could not HEAD request image URL. Using default extension .jpg for {image_url}")
                     extension = ".jpg" # Fallback if HEAD fails
            else:
                 logging.debug(f"Using extension '{extension}' from URL path.")


            final_filename = f"{base_filename}{extension}"
            save_path = save_directory / final_filename
            logging.info(f"Target save path: {save_path}")

            # Check if file already exists (optional, uncomment to enable skipping)
            # if save_path.exists():
            #     logging.info(f"Image already exists: {save_path}. Skipping download.")
            #     success = True # Consider existing file a success
            #     return success, tags, caption, str(save_path.name)

            logging.info(f"Downloading image: {image_url}...")
            img_response = requests.get(image_url, stream=True, headers=headers, timeout=45) # Longer timeout for download
            img_response.raise_for_status()

            with open(save_path, 'wb') as f:
                for chunk in img_response.iter_content(chunk_size=8192):
                    f.write(chunk)

            logging.info(f"Image download SUCCESS.")
            success = True

        except requests.exceptions.Timeout: logging.error(f"Image download timed out for {image_url}.")
        except requests.exceptions.HTTPError as e: logging.error(f"HTTP Error downloading image {image_url}: {e.response.status_code}")
        except requests.exceptions.RequestException as e: logging.error(f"Network error downloading image {image_url}: {e}")
        except IOError as e: logging.error(f"File IO error saving image {final_filename}: {e}")
        except Exception as e: logging.error(f"Unexpected error during image download/saving {image_url}: {e}", exc_info=False)


    except requests.exceptions.Timeout: logging.error(f"Request timed out fetching image page {page_url}.")
    except requests.exceptions.HTTPError as e: logging.error(f"HTTP Error fetching image page {page_url}: {e.response.status_code}")
    except requests.exceptions.RequestException as e: logging.error(f"Network error fetching image page {page_url}: {e}")
    except Exception as e: logging.error(f"Unexpected error processing image page {page_url}: {e}", exc_info=False)

    finally:
        # Return success status, metadata (even if download failed), and the intended/actual filename
        return success, tags, caption, final_filename


# --- Main Image Processing Function (MODIFIED: Creates/Quits its own driver) ---
def process_project_images(project_id: str, project_url: str) -> Tuple[bool, Optional[Path]]:
    """
    Processes images for a single ArchDaily project:
    Creates a temporary WebDriver, scrapes thumbnails, downloads images, saves image metadata JSON.

    Args:
        project_id (str): The ID of the project (used for folder/filenames).
        project_url (str): The URL of the ArchDaily project page.

    Returns:
        Tuple[bool, Optional[Path]]: Overall success status for image processing,
                                     and the Path object to the project folder.
    """
    logging.info(f"--- Processing Project Images for: {project_id} ({project_url}) ---")
    overall_image_success = False # Track if at least one image downloads
    base_download_folder = Path(project_id)
    driver = None # Initialize driver variable for this function scope

    try:
        # --- Set Up Base Folder ---
        logging.debug(f"Image download directory: '{base_download_folder.resolve()}'")
        base_download_folder.mkdir(parents=True, exist_ok=True)

        # --- Create Temporary WebDriver ---
        # This happens *only* if we need to process images
        driver = create_webdriver()
        if not driver:
            logging.error(f"Failed to create WebDriver for {project_id}. Cannot scrape thumbnails.")
            return False, base_download_folder # Indicate failure but return path

        # --- Scrape Thumbnails using the temporary driver ---
        logging.info(f"Scraping thumbnails...")
        success_scrape, gallery_items = scrape_gallery_thumbnails(driver, project_url)

        if not success_scrape:
            logging.error("Thumbnail scraping failed. Cannot proceed with image downloads.")
            return False, base_download_folder
        if not gallery_items:
            logging.info("No gallery items found to download.")
            return True, base_download_folder # No items is considered successful processing

        # --- Iterate and Download (using requests via download_gallery_image) ---
        logging.info(f"--- Starting image downloads for {len(gallery_items)} items ---")
        all_metadata = []
        successful_downloads = 0
        for index, item in enumerate(gallery_items):
            item_url = item.get('href')
            item_title = item.get('title', f'image_{index+1}') # Use sanitized title from scrape

            logging.info(f"\n>>> Downloading Item {index+1}/{len(gallery_items)}: '{item_title}' <<<")

            if not item_url:
                logging.warning("   Skipping: Missing URL.")
                continue

            # Use sanitized title for base filename, fallback to index
            # Further sanitize just in case, though scrape_gallery_thumbnails did basic pass
            safe_title_part = re.sub(r'[^a-zA-Z0-9_-]', '_', item_title)
            base_filename = f"{project_id}_{index+1:02d}_{safe_title_part[:50]}" # Add index and limit length

            # download_gallery_image uses requests, no driver needed here
            success_download, tags, caption, final_filename = download_gallery_image(
                item_url,
                base_download_folder,
                base_filename
            )

            if success_download and final_filename:
                logging.info(f"    SUCCESS: Saved as {final_filename}")
                metadata_entry = {
                    "filename": final_filename,
                    "original_title": item_title, # Store original title if needed
                    "tags": tags if tags else [],
                    "caption": caption if caption else ""
                }
                all_metadata.append(metadata_entry)
                successful_downloads += 1
                overall_image_success = True # Mark overall success if at least one downloads
            else:
                logging.warning(f"    FAILED to download or save image from {item_url}")
                # Optionally still add metadata even if download fails
                metadata_entry = {
                    "filename": None, # Indicate failed download
                    "original_title": item_title,
                    "tags": tags if tags else [],
                    "caption": caption if caption else "",
                    "source_url": item_url
                }
                all_metadata.append(metadata_entry)


            time.sleep(0.3) # Shorter delay, requests are less intensive than full page loads

        logging.info(f"\n--- Image download process complete: {successful_downloads}/{len(gallery_items)} successful. ---")

        # --- Save Image Metadata JSON ---
        if all_metadata:
            metadata_filepath = base_download_folder / f"{project_id}_images.json"
            logging.info(f"Saving image metadata to {metadata_filepath}")
            try:
                with open(metadata_filepath, 'w', encoding='utf-8') as f:
                    json.dump(all_metadata, f, indent=2, ensure_ascii=False) # Use indent=2
                logging.info("Image metadata saved.")
            except IOError as e:
                logging.error(f"Failed to save image metadata JSON: {e}")
        else:
            logging.info("No image metadata collected (perhaps scraping failed or no items).")

    except OSError as e:
        logging.error(f"Error creating/accessing image directory '{base_download_folder}': {e}.")
        return False, base_download_folder # Failed early
    except Exception as e:
        logging.error(f"An unexpected error occurred during image processing for {project_id}: {e}")
        # traceback.print_exc() # Enable for debugging
        overall_image_success = False # Ensure failure state on unexpected error
    finally:
        # --- Clean Up Temporary WebDriver ---
        if driver:
            logging.debug(f"Quitting WebDriver for project {project_id}...")
            try:
                driver.quit()
                logging.debug(f"WebDriver quit successfully for {project_id}.")
            except Exception as e:
                logging.error(f"Error quitting WebDriver for {project_id}: {e}")

    logging.info(f"--- Finished processing images for project {project_id}. Overall Success: {overall_image_success} ---")
    return overall_image_success, base_download_folder


# --- CSV Handling Functions (MODIFIED for Bulk Update) ---

# Removed update_csv_status as we do a bulk update at the end.
# Removed remove_csv_status as the main logic handles status checking.

def read_csv_data(csv_file: str) -> Tuple[Optional[List[str]], Optional[List[List[str]]]]:
    """Reads header and data rows from the CSV."""
    try:
        with open(csv_file, mode='r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            header = next(reader)
            rows = list(reader)
            return header, rows
    except FileNotFoundError:
        logging.error(f"CSV file not found: {csv_file}")
        return None, None
    except Exception as e:
        logging.error(f"Error reading CSV file {csv_file}: {e}")
        return None, None

def write_csv_data(csv_file: str, header: List[str], rows: List[List[str]]):
    """Writes the header and data rows to the CSV using a temporary file."""
    temp_file = Path(csv_file).with_suffix('.tmp')
    try:
        with open(temp_file, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(header)
            writer.writerows(rows)
        # Replace original file with temporary file
        os.replace(temp_file, csv_file)
        logging.info(f"Successfully updated CSV file: {csv_file}")
    except Exception as e:
        logging.error(f"Error writing updated CSV file {csv_file}: {e}")
        if temp_file.exists():
            try:
                os.remove(temp_file) # Clean up temp file on error
            except OSError:
                pass # Ignore cleanup error


# --- Worker Function for ThreadPool ---
def process_single_project(project_code: str, project_link: str) -> Tuple[str, str]:
    """
    Worker function executed by each thread.
    Processes one project (details + images) and returns its final status.
    """
    thread_name = threading.current_thread().name
    logging.info(f"Starting processing for project {project_code}")

    final_status = "error" # Default to error

    try:
        # --- Step 1: Scrape Project Details (using Requests) ---
        scraper_status, project_id = project_scraper(project_link) # Returns status and project_id

        # --- Step 2: Process Project Images (Conditionally, Creates own WebDriver) ---
        downloader_overall_success = False
        if scraper_status == "downloaded" and project_id:
            # process_project_images needs the project_id now
            downloader_overall_success, _ = process_project_images(project_id, project_link)
        elif scraper_status == "error":
             logging.warning(f"Skipping image download for {project_code} because project details scraping failed.")
        elif not project_id:
             logging.error(f"Skipping image download for {project_code} because project ID could not be determined.")
             scraper_status = "error" # Ensure status reflects ID failure

        # --- Step 3: Determine Final Status ---
        if scraper_status == "downloaded":
            # Check if *any* images were successfully downloaded OR if there were no images to download
            # process_project_images returns True if processing finished, even with 0 items
            # We need to distinguish between "all done" and "details done, images failed/skipped"
            project_dir = Path(project_id) if project_id else None
            images_json_path = project_dir / f"{project_id}_images.json" if project_dir else None

            if downloader_overall_success:
                 # This means either images were downloaded, or thumbnail scraping finished and found 0 items.
                 # Check if the _images.json exists to confirm image processing ran.
                 if images_json_path and images_json_path.exists():
                     final_status = "downloaded" # Details OK, Image processing ran (downloaded or found none)
                 else:
                     # This case might occur if thumbnail scraping failed AFTER details were downloaded
                     final_status = "incomplete" # Details OK, but image step didn't complete fully
            else:
                # Scraper OK, but downloader failed (e.g., WebDriver issue, download errors)
                final_status = "incomplete" # Details OK, Image processing failed
        else:
            # scraper_status was "error"
            final_status = "error"

        logging.info(f"Finished processing for project {project_code}. Final status: {final_status}")

    except Exception as e:
        logging.error(f"Unhandled exception processing project {project_code}: {e}")
        # traceback.print_exc() # Important for debugging worker errors
        final_status = "error" # Ensure error status on unexpected failure

    return project_code, final_status


# --- Main Execution Logic (Multithreaded) ---
if __name__ == "__main__":
    csv_file = './archdaily_projects.csv'
    base_url = 'https://www.archdaily.com'
    max_threads = 8 # Number of concurrent threads

    logging.info(f"Starting ArchDaily Scraper with {max_threads} threads.")

    # --- Read Initial CSV Data ---
    header, all_rows = read_csv_data(csv_file)
    if header is None or all_rows is None:
        exit("Failed to read CSV file. Exiting.")

    try:
        code_index = header.index('project_code') # Make sure column name matches your CSV
        status_index = header.index('status')
    except ValueError as e:
        logging.error(f"CSV header is missing required columns ('project_code' or 'status'): {e}. Header: {header}")
        exit("Exiting due to missing CSV columns.")

    # --- Filter Projects to Process ---
    projects_to_process: List[Tuple[str, str]] = []
    skip_statuses = {'downloaded', 'error', 'incomplete', 'duplicate', 'delete', 'manual'} # Lowercase set for efficient lookup
    projects_map = {row[code_index]: row for row in all_rows if len(row) > max(code_index, status_index)} # Map code to full row for later update

    logging.info(f"Total projects in CSV: {len(all_rows)}")

    for i, row in enumerate(all_rows):
         if not row or len(row) <= max(code_index, status_index):
             logging.warning(f"Skipping invalid or short row {i+1}: {row}")
             continue
         try:
            project_code = row[code_index]
            status = row[status_index].lower().strip()

            if status in skip_statuses:
                 # logging.debug(f"Skipping project {project_code} (Status: '{status}').") # Too verbose for info
                 continue
            else:
                 # Construct full project link
                 project_link = f"{base_url.strip('/')}/{project_code.strip('/')}"
                 projects_to_process.append((project_code, project_link))
                 logging.debug(f"Adding project to queue: {project_code}")

         except IndexError:
             logging.warning(f"Skipping row {i+1} due to index error (likely short row): {row}")
             continue


    logging.info(f"Found {len(projects_to_process)} projects to process.")

    if not projects_to_process:
        logging.info("No projects need processing based on current statuses. Exiting.")
        exit()

    # --- Process Projects using ThreadPoolExecutor ---
    results = {} # Dictionary to store {project_code: final_status}
    tasks_submitted = 0
    tasks_completed = 0

    # Using try/except/finally to ensure executor shutdown
    executor = None
    try:
        executor = concurrent.futures.ThreadPoolExecutor(max_workers=max_threads)
        future_to_project = {executor.submit(process_single_project, code, link): code for code, link in projects_to_process}
        tasks_submitted = len(future_to_project)
        logging.info(f"Submitted {tasks_submitted} tasks to the thread pool.")

        for future in concurrent.futures.as_completed(future_to_project):
            project_code = future_to_project[future]
            try:
                p_code, final_status = future.result() # Get result from worker
                results[p_code] = final_status
                tasks_completed += 1
                logging.info(f"Completed: {tasks_completed}/{tasks_submitted} - Project {p_code} -> {final_status}")
            except Exception as exc:
                logging.error(f'Project {project_code} generated an exception: {exc}')
                # traceback.print_exc() # Log the full traceback for failed futures
                results[project_code] = 'error' # Mark as error if future failed
                tasks_completed += 1 # Still count it as completed task attempt
                logging.info(f"Completed (with error): {tasks_completed}/{tasks_submitted} - Project {project_code} -> error")


    except Exception as e:
         logging.error(f"An error occurred during thread pool execution: {e}")
         # traceback.print_exc()
    finally:
        if executor:
            logging.info("Shutting down thread pool executor...")
            executor.shutdown(wait=True) # Wait for running threads to finish
            logging.info("Thread pool shutdown complete.")


    # --- Bulk Update CSV with Results ---
    logging.info("\nUpdating CSV file with final statuses...")
    if not results:
        logging.info("No results collected, skipping CSV update.")
    else:
        updated_rows = []
        updated_count = 0
        for row_data in all_rows:
            # Ensure row has enough columns before trying to access/update
            if len(row_data) > max(code_index, status_index):
                p_code = row_data[code_index]
                if p_code in results:
                    new_status = results[p_code]
                    if row_data[status_index] != new_status:
                        row_data[status_index] = new_status # Update status in the row list
                        updated_count += 1
                        logging.debug(f"Marking {p_code} as {new_status}")
                updated_rows.append(row_data)
            else:
                 # Keep short/invalid rows as they were, but log it
                 logging.warning(f"Keeping original short/invalid row: {row_data}")
                 updated_rows.append(row_data)


        # Write the modified data back to CSV
        write_csv_data(csv_file, header, updated_rows)
        logging.info(f"CSV update finished. {updated_count} statuses were changed.")

    logging.info("\nScript finished.")

2025-04-07 00:27:52,329 - MainThread - INFO - Starting ArchDaily Scraper with 8 threads.
2025-04-07 00:27:52,331 - MainThread - ERROR - CSV header is missing required columns ('project_code' or 'status'): 'project_code' is not in list. Header: ['project_id', 'link', 'keyword', 'status']
2025-04-07 00:27:52,332 - MainThread - INFO - Total projects in CSV: 8
2025-04-07 00:27:52,332 - MainThread - INFO - Found 8 projects to process.
2025-04-07 00:27:52,333 - ThreadPoolExecutor-1_0 - INFO - Starting processing for project 904658
2025-04-07 00:27:52,334 - ThreadPoolExecutor-1_1 - INFO - Starting processing for project 965203
2025-04-07 00:27:52,335 - ThreadPoolExecutor-1_2 - INFO - Starting processing for project 343719
2025-04-07 00:27:52,335 - ThreadPoolExecutor-1_0 - INFO - --- Scraping Project Details for: https://www.archdaily.com/904658 ---
2025-04-07 00:27:52,335 - ThreadPoolExecutor-1_3 - INFO - Starting processing for project 563604
2025-04-07 00:27:52,337 - ThreadPoolExecutor-1_4 